# **in this notebook I followed an exploratory approach to see which methods work best for anomaly detection**

the reason for the weak results is a problem with the dataset temporal relations and generation logic  I plan to fix that for the finals as well as trying a few more methods that might work better

# **Upload and explore dataset**








In [ ]:
#uploading dataset
try:
    from google.colab import files
    uploaded = files.upload()
    csv_name = next(iter(uploaded))
except Exception:
    csv_name = "linux_auth_logs_labeled.csv"

print('csv_name set to:', csv_name)

Saving linux_auth_logs_full(new).csv to linux_auth_logs_full(new).csv
csv_name set to: linux_auth_logs_full(new).csv


In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, average_precision_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


import random
np.random.seed(42)
random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

import joblib

%matplotlib inline

In [ ]:
#read the dataset and pring the head
df = pd.read_csv(csv_name)

df.head()

,timestamp,source_ip,server,username,service,attempts,status,port,protocol,comment,anomaly_label,delta_t
0,2023-01-01 00:42:39.936864,175.59.175.26,web-sgp-02,aaron54,login,1,Failed,22.0,SSH2,User aaron54 login,normal,NaN
1,2023-01-01 00:42:44.872570,68.162.133.242,db-ber-01,aaron54,ssh,1,Failed,22.0,SSH2,User aaron54 login,normal,4.935706
2,2023-01-01 00:42:45.001529,70.154.139.216,srv-nyc-01,aaron54,cron,1,Failed,22.0,SSH2,User aaron54 login,normal,0.128959
3,2023-01-01 00:42:45.096641,40.138.230.209,srv-ldn-02,aaron54,login,1,Success,22.0,SSH2,User aaron54 login,normal,0.095112
4,2023-01-01 00:42:45.122138,9.151.194.9,web-sgp-02,aaron54,cron,1,Success,22.0,SSH2,User aaron54 login,normal,0.025497


In [ ]:
#dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   timestamp      500000 non-null  object 
 1   source_ip      500000 non-null  object 
 2   server         500000 non-null  object 
 3   username       500000 non-null  object 
 4   service        500000 non-null  object 
 5   attempts       500000 non-null  int64  
 6   status         500000 non-null  object 
 7   port           475000 non-null  float64
 8   protocol       415000 non-null  object 
 9   comment        500000 non-null  object 
 10  anomaly_label  500000 non-null  object 
 11  delta_t        496987 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 45.8+ MB


#**Preprocessing the Dataset**

In [ ]:
# Convert the 'timestamp' column in the original df to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])



In [ ]:
df['anomaly_label'].value_counts()

,count
anomaly_label,
normal,350000
port_scan,60000
brute_force,50000
privilege_escalation,25000
geo_anomaly,15000


#Brute-force Attacks data prep

In [ ]:
# Sort df_unlabeled by 'username' and 'timestamp'
df_unlabeled = df_unlabeled.sort_values(['username', 'timestamp'])

print(df_unlabeled.dtypes)
df_unlabeled.head(10)

NameError: name 'df_unlabeled' is not defined

In [ ]:
# User-level aggregated features
user_features = df.groupby('username').agg({
    'timestamp': ['count', 'min', 'max'],  # Total activities, first & last login
    'status': lambda x: (x == 'Failed').sum(),  # Total failures
    'attempts': ['sum', 'mean', 'max'],  # Attempt patterns
    'source_ip': 'nunique',  # Unique IPs used
    'service': 'nunique',  # Unique services accessed
    'server': 'nunique',  # Unique servers accessed
    'port': 'nunique'  # Unique ports used
}).reset_index()

# Flatten the column names
user_features.columns = ['username', 'total_activities', 'first_activity', 'last_activity',
                        'total_failures', 'total_attempts', 'avg_attempts', 'max_attempts',
                        'unique_ips', 'unique_services', 'unique_servers', 'unique_ports']

# Calculate derived features
user_features['failure_rate'] = user_features['total_failures'] / user_features['total_activities']
user_features['activity_duration_days'] = (user_features['last_activity'] - user_features['first_activity']).dt.total_seconds() / (24 * 3600)
user_features['activities_per_day'] = user_features['total_activities'] / user_features['activity_duration_days'].replace(0, 1)  # Avoid division by zero

In [ ]:
print(user_features.head(5))

#Geolocation Attacks data prep

In [ ]:
# Sort by username and timestamp
df_geo = df_unlabeled.sort_values(['username', 'timestamp']).copy()

# Calculate time differences between consecutive logins
df_geo['time_since_last_login'] = df_geo.groupby('username')['timestamp'].diff()
df_geo['time_since_last_login_hours'] = df_geo['time_since_last_login'].dt.total_seconds() / 3600

# Server and IP access patterns (count occurrences)
df_geo['server_access_count'] = df_geo.groupby(['username', 'server'])['server'].transform('count')
df_geo['ip_access_count'] = df_geo.groupby(['username', 'source_ip'])['source_ip'].transform('count')

# Create change flags (these will be boolean -> can be summed)
df_geo['server_change'] = (df_geo.groupby('username')['server'].shift() != df_geo['server']).fillna(False)
df_geo['ip_change'] = (df_geo.groupby('username')['source_ip'].shift() != df_geo['source_ip']).fillna(False)

# Consecutive same location count
df_geo['location_group'] = (df_geo['server_change'] | df_geo['ip_change']).cumsum()
df_geo['consecutive_same_location'] = df_geo.groupby(['username', 'location_group']).cumcount() + 1

print("Processed data sample:")
print(df_geo[['username', 'timestamp', 'source_ip', 'server', 'time_since_last_login_hours', 'server_change']].head(10))





In [ ]:
# Separate aggregations by data type
basic_features = df_geo.groupby('username').agg({
    'server': 'nunique',
    'source_ip': 'nunique',
    'service': 'nunique',
    'timestamp': ['count', 'min', 'max'],
}).reset_index()
basic_features.columns = ['username', 'unique_servers', 'unique_ips', 'unique_services', 'total_logins', 'first_seen', 'last_seen']

# For numeric features - handle NaN during aggregation
numeric_features = df_geo.groupby('username').agg({
    'time_since_last_login_hours': ['mean', 'std', 'min', 'max'],
    'server_access_count': ['mean', 'max'],
    'ip_access_count': ['mean', 'max'],
    'attempts': ['mean', 'max', 'sum'],
    'port': 'nunique',
}).reset_index()
numeric_features.columns = ['username', 'avg_time_between_logins', 'std_time_between_logins', 'min_time_between_logins',
                           'max_time_between_logins', 'avg_server_access', 'max_server_access', 'avg_ip_access',
                           'max_ip_access', 'avg_attempts', 'max_attempts', 'total_attempts', 'unique_ports']

# Boolean features
boolean_features = df_geo.groupby('username').agg({
    'server_change': 'sum',
    'ip_change': 'sum',
    'consecutive_same_location': 'max',
}).reset_index()
boolean_features.columns = ['username', 'server_changes', 'ip_changes', 'max_consecutive_same_location']

# Status features
status_features = df_geo.groupby('username').agg({
    'status': lambda x: (x == 'failure').sum(),
}).reset_index()
status_features.columns = ['username', 'failed_logins']

# Merge all features
geo_features = basic_features.merge(numeric_features, on='username', how='left')
geo_features = geo_features.merge(boolean_features, on='username', how='left')
geo_features = geo_features.merge(status_features, on='username', how='left')

#handle devision by zero
geo_features['active_days'] = (
    (geo_features['last_seen'] - geo_features['first_seen']).dt.total_seconds() / (24 * 3600)
).replace(0, 1)  # Avoid division by zero

# Safe division to prevent NaN
geo_features['logins_per_day'] = np.where(
    geo_features['active_days'] > 0,
    geo_features['total_logins'] / geo_features['active_days'],
    0
)
geo_features['servers_per_login'] = np.where(
    geo_features['total_logins'] > 0,
    geo_features['unique_servers'] / geo_features['total_logins'],
    0
)
geo_features['ips_per_login'] = np.where(
    geo_features['total_logins'] > 0,
    geo_features['unique_ips'] / geo_features['total_logins'],
    0
)
geo_features['failure_rate'] = np.where(
    geo_features['total_logins'] > 0,
    geo_features['failed_logins'] / geo_features['total_logins'],
    0
)

# Handle NaN in std_time_between_logins (occurs when users have only one login)
geo_features['std_time_between_logins'] = geo_features['std_time_between_logins'].fillna(0)

# Risk scores
geo_features['location_volatility'] = np.where(
    geo_features['total_logins'] > 0,
    (geo_features['server_changes'] + geo_features['ip_changes']) / geo_features['total_logins'],
    0
)

geo_features['geo_risk_score'] = (
    geo_features['unique_ips'] * 0.3 +
    geo_features['unique_servers'] * 0.3 +
    geo_features['location_volatility'] * 0.2 +
    geo_features['failure_rate'] * 0.2
)

# Fill any remaining NaN values with median
numeric_cols = geo_features.select_dtypes(include=np.number).columns
geo_features[numeric_cols] = geo_features[numeric_cols].fillna(geo_features[numeric_cols].median())

print(f"\nRemaining NaN values in geo_features: {geo_features.isnull().sum().sum()}")


#Port Scanning Attack data prep

In [ ]:
# For port scanning we want the DataFrame sorted by source_ip and timestamp
df_by_source = df.sort_values(['source_ip', 'timestamp'])

In [ ]:
# Add a 15-minute floor time window column on the df_by_source DataFrame
df_by_source['time_window'] = df_by_source['timestamp'].dt.floor('15min')

In [ ]:
# Aggregate port scanning features per source_ip/time_window using df_by_source
port_scan_features = df_by_source.groupby(['source_ip', 'time_window']).agg({
    'port': [
        'count',  # total_port_attempts
        'nunique',  # unique_ports_scanned
        lambda x: (x > 10000).sum(),  # high_port_attempts (verify threshold)
        'min',  # min_port
        'max',  # max_port
        lambda x: np.std(x) if len(x) > 1 else 0,  # port_std_dev
    ],
    'status': [
        lambda x: (x == 'Failed').sum(),  # failed_attempts
        lambda x: (x == 'Success').sum(),  # success_attempts
    ],
    'server': 'nunique',  # unique_servers_targeted
    'service': 'nunique',  # unique_services_accessed
    'username': 'nunique',  # unique_usernames_tried
}).reset_index()


In [ ]:
port_scan_features.columns = [
    'source_ip', 'time_window',
    'total_port_attempts', 'unique_ports_scanned', 'high_port_attempts',
    'min_port', 'max_port', 'port_std_dev',
    'failed_attempts', 'success_attempts',
    'unique_servers', 'unique_services', 'unique_usernames'
]

In [ ]:
port_scan_features['port_range'] = port_scan_features['max_port'] - port_scan_features['min_port']
port_scan_features['high_port_ratio'] = port_scan_features['high_port_attempts'] / port_scan_features['total_port_attempts']
port_scan_features['failure_rate'] = port_scan_features['failed_attempts'] / port_scan_features['total_port_attempts']
port_scan_features['success_rate'] = port_scan_features['success_attempts'] / port_scan_features['total_port_attempts']
port_scan_features['ports_per_minute'] = port_scan_features['unique_ports_scanned'] / 15  # 15-minute windows


In [ ]:
port_scan_features = port_scan_features.replace([np.inf, -np.inf], 0).fillna(0)

print("Port scanning features shape:", port_scan_features.shape)
print("\nFeatures created:")
print(port_scan_features.columns.tolist())
print("\nSample of engineered features:")
print(port_scan_features.head())

#Privilage Escilation data prep

In [ ]:
df_by_user = df.sort_values(['username', 'timestamp'])

df_by_user.head(10)

In [ ]:

priv_keywords = [
    "sudo", "su -", "root", "chmod 777", "chown", "setuid", "setgid",
    "privilege", "escalate", "elevate", "exploit", "admin access",
    "bypass", "shell", "spawn", "superuser", "policy"
]

def has_priv_keyword(text):
    # Handle NaNs and non-string values safely
    if not isinstance(text, str) or not text:
        return False
    text = text.lower()
    return any(k in text for k in priv_keywords)


priv_user_features = df_by_user.groupby('username').agg({
    'timestamp': ['count', 'min', 'max'],
    'comment': lambda x: sum(has_priv_keyword(t) for t in x),
    'service': 'nunique',
    'server': 'nunique',
    'attempts': ['sum', 'mean', 'max']}).reset_index()

In [ ]:

priv_user_features.columns = [
    'username',
    'total_activities', 'first_activity', 'last_activity',
    'priv_keyword_count',
    'unique_services', 'unique_servers',
    'total_attempts', 'avg_attempts', 'max_attempts',
]


priv_user_features['priv_keyword_rate'] = (
    priv_user_features['priv_keyword_count'] / priv_user_features['total_activities']
)


priv_user_features['activity_duration_days'] = (
    (priv_user_features['last_activity'] - priv_user_features['first_activity'])
    .dt.total_seconds() / (24 * 3600)
)


priv_user_features['activities_per_day'] = (
    priv_user_features['total_activities'] /
    priv_user_features['activity_duration_days'].replace(0, 1)
)

In [ ]:
print(priv_user_features.head(10))
print(priv_user_features.describe())


print(priv_user_features.sort_values('priv_keyword_count', ascending=False).head(10))

#Scaling data for PCA



In [ ]:
# Identify numerical columns to be used for scaling and PCA
numerical_cols_bf = user_features.select_dtypes(include=np.number).columns.tolist()

# Exclude 'username' and timestamp columns
exclude_cols = ['username', 'first_activity', 'last_activity']
bf_features_for_scaling = [col for col in numerical_cols_bf if col not in exclude_cols]

# Select only the features for scaling
X_bf_numerical = user_features[bf_features_for_scaling]

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical features
scaled_bf_features = scaler.fit_transform(X_bf_numerical)

print("Shape of scaled brute force features:", scaled_bf_features.shape)
print("First 5 rows of scaled brute force features:\n", scaled_bf_features[:5])

# --- PCA for brute-force features ---
from sklearn.decomposition import PCA
pca_bf = PCA(n_components=0.95)  # retain 95% variance
pca_bf_features = pca_bf.fit_transform(scaled_bf_features)
pca_bf_df = pd.DataFrame(pca_bf_features, columns=[f'PC_{i+1}' for i in range(pca_bf_features.shape[1])])
print(f"Number of components for Brute-force PCA: {pca_bf.n_components_}")
print(f"Explained variance ratio for Brute-force PCA: {pca_bf.explained_variance_ratio_.sum():.2f}")
print("\nFirst 5 rows of PCA transformed Brute-force features:")
print(pca_bf_df.head())

In [ ]:
# Prepare numeric geolocation features and scale them before PCA
numerical_cols_geo = geo_features.select_dtypes(include=np.number).columns.tolist()
exclude_cols_geo = ['username', 'first_seen', 'last_seen']
geo_features_for_scaling = [c for c in numerical_cols_geo if c not in exclude_cols_geo]
X_geo_numerical = geo_features[geo_features_for_scaling]
scaler_geo = StandardScaler()
scaled_geo_features = scaler_geo.fit_transform(X_geo_numerical)

# Initialize PCA for geolocation features
pca_geo = PCA(n_components=0.95) # Retain 95% of variance

# Fit PCA to the scaled data and transform it
pca_geo_features = pca_geo.fit_transform(scaled_geo_features)

# Create a DataFrame from the PCA transformed features for better readability
pca_geo_df = pd.DataFrame(pca_geo_features, columns=[f'PC_{i+1}' for i in range(pca_geo_features.shape[1])])

print(f"Number of components for Geolocation PCA: {pca_geo.n_components_}")
print(f"Explained variance ratio for Geolocation PCA: {pca_geo.explained_variance_ratio_.sum():.2f}")
print("\nFirst 5 rows of PCA transformed Geolocation features:")
print(pca_geo_df.head())

In [ ]:
numerical_cols_ps = port_scan_features.select_dtypes(include=np.number).columns.tolist()


exclude_cols_ps = ['source_ip', 'time_window']
ps_features_for_scaling = [col for col in numerical_cols_ps if col not in exclude_cols_ps]


X_ps_numerical = port_scan_features[ps_features_for_scaling]

scaler_ps = StandardScaler()

scaled_ps_features = scaler_ps.fit_transform(X_ps_numerical)

print("Shape of scaled port scanning features:", scaled_ps_features.shape)
print("First 5 rows of scaled port scanning features:\n", scaled_ps_features[:5])

In [ ]:
pca_ps = PCA(n_components=0.95)


pca_ps_features = pca_ps.fit_transform(scaled_ps_features)

pca_ps_df = pd.DataFrame(pca_ps_features, columns=[f'PC_{i+1}' for i in range(pca_ps_features.shape[1])])

print(f"Number of components for Port Scanning PCA: {pca_ps.n_components_}")
print(f"Explained variance ratio for Port Scanning PCA: {pca_ps.explained_variance_ratio_.sum():.2f}")
print("\nFirst 5 rows of PCA transformed Port Scanning features:")
print(pca_ps_df.head())

In [ ]:
numerical_cols_priv = priv_user_features.select_dtypes(include=np.number).columns.tolist()


exclude_cols_priv = ['username', 'first_activity', 'last_activity']
priv_features_for_scaling = [col for col in numerical_cols_priv if col not in exclude_cols_priv]


X_priv_numerical = priv_user_features[priv_features_for_scaling]


scaler_priv = StandardScaler()


scaled_priv_features = scaler_priv.fit_transform(X_priv_numerical)

print("Shape of scaled privilege escalation features:", scaled_priv_features.shape)
print("First 5 rows of scaled privilege escalation features:\n", scaled_priv_features[:5])

In [ ]:
pca_priv = PCA(n_components=0.95)


pca_priv_features = pca_priv.fit_transform(scaled_priv_features)


pca_priv_df = pd.DataFrame(pca_priv_features, columns=[f'PC_{i+1}' for i in range(pca_priv_features.shape[1])])

print(f"Number of components for Privilege Escalation PCA: {pca_priv.n_components_}")
print(f"Explained variance ratio for Privilege Escalation PCA: {pca_priv.explained_variance_ratio_.sum():.2f}")
print("\nFirst 5 rows of PCA transformed Privilege Escalation features:")
print(pca_priv_df.head())

#Feature Selection using
* heuristic selection
* variance based selection
* PCA-based selection


In [ ]:
def select_features_by_attack_type(attack_type):
    """Return a heuristic list of columns to keep for the given attack type.
    This is based on the engineered feature DataFrames created previously:
    - 'brute_force' -> `user_features`
    - 'geolocation' -> `geo_features`
    - 'port_scan' -> `port_scan_features`
    - 'privilege' -> `priv_user_features`
    """
    attack_type = attack_type.lower()
    if attack_type == 'brute_force' or attack_type == 'bruteforce':
        return [
            'total_activities', 'total_failures', 'failure_rate',
            'avg_attempts', 'max_attempts', 'activities_per_day',
            'unique_ips'
        ]
    elif attack_type == 'geolocation' or attack_type == 'geo':
        return [
            'unique_ips', 'unique_servers', 'unique_services',
            'logins_per_day', 'active_days', 'geo_risk_score'
        ]
    elif attack_type == 'port_scan' or attack_type == 'port_scanning':
        return [
            'total_port_attempts', 'unique_ports_scanned', 'ports_per_minute',
            'port_range', 'high_port_ratio', 'unique_services', 'unique_usernames'
        ]
    elif attack_type == 'privilege' or attack_type == 'priv_escalation':
        return [
            'priv_keyword_count', 'priv_keyword_rate', 'activities_per_day',
            'unique_services', 'unique_servers'
        ]
    else:

        return None

def variance_based_selection(df, top_k=10, corr_threshold=0.9):
    """Select top_k features by variance, then drop highly correlated features.
    Returns ordered list of selected columns.
    """

    num = df.select_dtypes(include=[np.number]).copy()
    if num.shape[1] == 0:
        return []
    variances = num.var().sort_values(ascending=False)
    top_cols = variances.head(top_k).index.tolist()


    keep = []
    corr = num[top_cols].corr().abs()
    for col in top_cols:
        if not any((corr.loc[col, other] > corr_threshold) for other in keep):
            keep.append(col)
    return keep

def pca_loadings_selection(df, n_components=5, top_k=10):
    """Use PCA loadings to pick features that have large absolute weights
    across the first `n_components` principal components.
    Returns a list of selected column names.
    """
    num = df.select_dtypes(include=[np.number]).copy()
    if num.shape[1] == 0:
        return []

    scaler_tmp = StandardScaler()
    X = scaler_tmp.fit_transform(num.fillna(0))
    pca_tmp = PCA(n_components=min(n_components, X.shape[1]))
    pca_tmp.fit(X)
    loadings = np.abs(pca_tmp.components_).sum(axis=0)
    feature_scores = pd.Series(loadings, index=num.columns)
    selected = feature_scores.sort_values(ascending=False).head(top_k).index.tolist()
    return selected

def prepare_selected_features(df, selected_cols, scaler=None):
    """Given a DataFrame and a list of columns, return scaled numpy array
    ready for anomaly models and the fitted scaler.
    """
    if selected_cols is None:

        selected_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[selected_cols].fillna(0).astype(float)
    if scaler is None:
        scaler = StandardScaler()
        Xs = scaler.fit_transform(X)
    else:
        Xs = scaler.transform(X)
    return Xs, scaler, selected_cols

In [ ]:
# Clear everything and start fresh
import pandas as pd
import numpy as np

print("=== STARTING FRESH WITH CONSISTENT DATA ===")

# 1. Make sure df is your 500K dataset
print(f"Using df with {df.shape[0]:,} rows, {df.shape[1]} columns")

# 2. Reset all feature DataFrames to df
user_features = df.copy()
geo_features = df.copy()
priv_user_features = df.copy()
port_scan_features = df.copy()

print(f"\n✅ All feature DataFrames now have {df.shape[0]:,} rows")

# 3. Recreate ground truth from df
print("\nCreating ground truth labels from df...")
user_ground_truth_df = pd.DataFrame({
    'username': df['username'],
    'ground_truth': df['anomaly_label'].map({
        'normal': 0,
        'brute_force': 1,
        'geo_anomaly': 2,
        'privilege_escalation': 3,
        'port_scan': 0
    })
})

port_scan_ground_truth_df = pd.DataFrame({
    'source_ip': df['source_ip'],
    'time_window': df['timestamp'].dt.floor('H'),
    'ground_truth': (df['anomaly_label'] == 'port_scan').astype(int)
})

print(f"user_ground_truth_df: {user_ground_truth_df.shape}")
print(f"Brute force anomalies: {(user_ground_truth_df['ground_truth'] == 1).sum():,}")
print(f"Port scan anomalies: {port_scan_ground_truth_df['ground_truth'].sum():,}")

# 4. Now run your feature selection code (the block you posted)
print("\n" + "="*60)
print("Running feature selection with consistent data...")
print("="*60)

# Clear old results
results = {}

# Run your feature selection code (it will now use consistent 500K data)
# ... paste your feature selection code here ...

# 5. THEN run the stratified split
print("\n" + "="*60)
print("Running stratified splits...")
print("="*60)

# Run the rebuild_stratified_splits() function
results = rebuild_stratified_splits()

print("\n✅ DONE! You should now have thousands of anomalies per split.")

In [ ]:

persist = {k: {
    'heuristic_cols': v['heuristic_cols'],
    'variance_cols': v['variance_cols'],
    'pca_cols': v['pca_cols']
} for k, v in results.items()}
with open('selected_feature_sets.json', 'w') as f:
    json.dump(persist, f, indent=2)
print('Saved selected_feature_sets.json')

#Splitting the dataset and training prep

In [ ]:
# === STRATIFIED SPLIT REBUILD - FIXED VERSION ===
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import warnings

def rebuild_stratified_splits():
    """
    CORRECT stratified splitting that preserves rare anomaly samples
    """
    global results

    rebuild_count = 0
    print("Rebuilding splits with PROPER stratified sampling...")

    for attack_type in ['brute_force', 'geolocation', 'privilege', 'port_scan']:
        print(f"\n{'='*60}")
        print(f"Processing: {attack_type}")
        print('='*60)

        # Get the CORRECT base dataframe and labels
        if attack_type == 'brute_force':
            base_df = user_features
            # Brute force labels: 1 if brute_force, 0 otherwise
            y_full = (user_ground_truth_df['ground_truth'] == 1).astype(int).values

        elif attack_type == 'geolocation':
            base_df = geo_features
            # Geo anomaly labels: 1 if geo_anomaly, 0 otherwise
            y_full = (user_ground_truth_df['ground_truth'] == 2).astype(int).values

        elif attack_type == 'privilege':
            base_df = priv_user_features
            # Privilege escalation labels: 1 if privilege_escalation, 0 otherwise
            y_full = (user_ground_truth_df['ground_truth'] == 3).astype(int).values

        elif attack_type == 'port_scan':
            base_df = port_scan_features
            # Port scan labels: 1 if port_scan, 0 otherwise
            y_full = (port_scan_ground_truth_df['ground_truth'] == 1).astype(int).values

        # Check anomaly count
        n_anomalies = np.sum(y_full == 1)
        n_total = len(y_full)
        anomaly_rate = n_anomalies / n_total

        print(f"Total samples: {n_total:,}")
        print(f"Anomalies: {n_anomalies:,} ({anomaly_rate:.4%})")

        # If too few anomalies, use manual split instead of stratified
        if n_anomalies < 10:
            print(f"⚠️  Too few anomalies ({n_anomalies}) - using MANUAL split")
            # Use manual split function (defined below)
            train_idx, val_idx, test_idx = manual_split_for_rare_class(y_full, min_anomalies=5)
        else:
            # Use STRATIFIED split
            print(f"✅ Using stratified split")

            # Split 1: Train+Val (80%) vs Test (20%)
            sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
            train_val_idx, test_idx = next(sss1.split(np.zeros(len(y_full)), y_full))

            # Split 2: Train (75% of train_val) vs Val (25% of train_val)
            sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
            temp_y = y_full[train_val_idx]
            train_temp_idx, val_temp_idx = next(sss2.split(np.zeros(len(temp_y)), temp_y))

            train_idx = train_val_idx[train_temp_idx]
            val_idx = train_val_idx[val_temp_idx]

        # Create splits
        final_train_idx = train_idx
        final_val_idx = val_idx
        final_test_idx = test_idx

        # Verify anomaly distribution
        train_anom = np.sum(y_full[final_train_idx] == 1)
        val_anom = np.sum(y_full[final_val_idx] == 1)
        test_anom = np.sum(y_full[final_test_idx] == 1)

        print(f"Train: {len(final_train_idx):,} samples, {train_anom} anomalies")
        print(f"Val:   {len(final_val_idx):,} samples, {val_anom} anomalies")
        print(f"Test:  {len(final_test_idx):,} samples, {test_anom} anomalies")

        # For each feature selection method
        for sel_label, col_key in [('heuristic', 'heuristic_cols'),
                                   ('variance', 'variance_cols'),
                                   ('pca', 'pca_cols')]:

            # Get selected columns
            if attack_type not in results or col_key not in results[attack_type]:
                print(f"  Skipping {sel_label}: no columns found")
                continue

            sel_cols = results[attack_type][col_key]
            if not sel_cols or len(sel_cols) == 0:
                print(f"  Skipping {sel_label}: empty column list")
                continue

            # Prepare features for ALL data first
            X_full, scaler_sel, used_cols = prepare_selected_features(base_df, sel_cols, scaler=None)

            # Create splits by indexing
            X_train = X_full[final_train_idx]
            X_val = X_full[final_val_idx]
            X_test = X_full[final_test_idx]
            y_train = y_full[final_train_idx]
            y_val = y_full[final_val_idx]
            y_test = y_full[final_test_idx]

            # Store in results
            if attack_type not in results:
                results[attack_type] = {}

            results[attack_type][f'{sel_label}_X_train'] = X_train
            results[attack_type][f'{sel_label}_X_val'] = X_val
            results[attack_type][f'{sel_label}_X_test'] = X_test
            results[attack_type][f'{sel_label}_y_train'] = y_train
            results[attack_type][f'{sel_label}_y_val'] = y_val
            results[attack_type][f'{sel_label}_y_test'] = y_test
            results[attack_type][f'{sel_label}_scaler'] = scaler_sel

            rebuild_count += 1

            print(f"  ✓ {sel_label}: Train={X_train.shape}, Anom={y_train.sum()}")

    print(f"\n{'='*60}")
    print(f"✅ Rebuild complete. Updated {rebuild_count} splits.")
    print('='*60)
    return results

def manual_split_for_rare_class(y, min_anomalies=5):
    """
    Manual split when stratified can't work (too few anomalies)
    """
    # Find anomaly indices
    anomaly_idx = np.where(y == 1)[0]
    normal_idx = np.where(y == 0)[0]

    # Shuffle
    np.random.shuffle(anomaly_idx)
    np.random.shuffle(normal_idx)

    # Split anomalies: 70% train, 15% val, 15% test
    n_anom = len(anomaly_idx)
    n_train_anom = int(0.7 * n_anom)
    n_val_anom = int(0.15 * n_anom)

    # Ensure minimum anomalies
    n_train_anom = max(n_train_anom, min_anomalies)
    n_val_anom = max(n_val_anom, min(2, n_anom - n_train_anom))
    n_test_anom = n_anom - n_train_anom - n_val_anom

    # Split normals: same ratios
    n_norm = len(normal_idx)
    n_train_norm = int(0.7 * n_norm)
    n_val_norm = int(0.15 * n_norm)

    # Create indices
    train_idx = np.concatenate([
        anomaly_idx[:n_train_anom],
        normal_idx[:n_train_norm]
    ])

    val_idx = np.concatenate([
        anomaly_idx[n_train_anom:n_train_anom + n_val_anom],
        normal_idx[n_train_norm:n_train_norm + n_val_norm]
    ])

    test_idx = np.concatenate([
        anomaly_idx[n_train_anom + n_val_anom:],
        normal_idx[n_train_norm + n_val_norm:]
    ])

    # Shuffle each set
    np.random.shuffle(train_idx)
    np.random.shuffle(val_idx)
    np.random.shuffle(test_idx)

    return train_idx, val_idx, test_idx

# Run the fix
results = rebuild_stratified_splits()

# Verify the splits
print("\n" + "="*60)
print("VERIFICATION - Anomaly counts per split:")
print("="*60)

summary_data = []
for attack_type in results:
    for prefix in ['heuristic', 'variance', 'pca']:
        y_train = results[attack_type].get(f'{prefix}_y_train')
        y_val = results[attack_type].get(f'{prefix}_y_val')
        y_test = results[attack_type].get(f'{prefix}_y_test')

        if y_train is not None:
            summary_data.append({
                'attack_type': attack_type,
                'prefix': prefix,
                'n_train': len(y_train),
                'n_val': len(y_val) if y_val is not None else 0,
                'n_test': len(y_test) if y_test is not None else 0,
                'anoms_train': int(y_train.sum()),
                'anoms_val': int(y_val.sum()) if y_val is not None else 0,
                'anoms_test': int(y_test.sum()) if y_test is not None else 0
            })

# Create summary DataFrame
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string())

In [ ]:
# Assuming df is your 500,000 row generated dataset
print(f"Total dataset shape: {df.shape}")
print(f"\nAnomaly distribution:")
print(df['anomaly_label'].value_counts())
print(df['anomaly_label'].value_counts(normalize=True))

# You should see something like:
# normal                  350000  (70%)
# port_scan                60000  (12%)
# brute_force              50000  (10%)
# privilege_escalation     25000  (5%)
# geo_anomaly              15000  (3%)

In [ ]:
print("=== DEBUGGING THE DATA PIPELINE ===")

# 1. Check your feature DataFrames
print("\n1. Feature DataFrame sizes:")
print(f"user_features shape: {user_features.shape if 'user_features' in globals() else 'NOT FOUND'}")
print(f"geo_features shape: {geo_features.shape if 'geo_features' in globals() else 'NOT FOUND'}")
print(f"priv_user_features shape: {priv_user_features.shape if 'priv_user_features' in globals() else 'NOT FOUND'}")
print(f"port_scan_features shape: {port_scan_features.shape if 'port_scan_features' in globals() else 'NOT FOUND'}")

# 2. Check ground truth DataFrames
print("\n2. Ground truth DataFrame sizes:")
print(f"user_ground_truth_df shape: {user_ground_truth_df.shape if 'user_ground_truth_df' in globals() else 'NOT FOUND'}")
print(f"port_scan_ground_truth_df shape: {port_scan_ground_truth_df.shape if 'port_scan_ground_truth_df' in globals() else 'NOT FOUND'}")

# 3. Check if they match df
print("\n3. Comparing to df (500K):")
if 'user_ground_truth_df' in globals():
    print(f"user_ground_truth_df rows: {user_ground_truth_df.shape[0]:,}")
    print(f"df rows: {df.shape[0]:,}")
    if user_ground_truth_df.shape[0] == df.shape[0]:
        print("✅ MATCH!")
    else:
        print(f"🚨 MISMATCH! Difference: {df.shape[0] - user_ground_truth_df.shape[0]:,} rows")

In [ ]:
# Drop the 'anomaly_label' column to create df_unlabeled
df_unlabeled = df.drop('anomaly_label', axis=1)

In [ ]:
# DEBUG: Check what's actually in results
print("DEBUG: Checking results structure...")
for attack_type in results.keys():
    print(f"\n{attack_type}:")
    print(f"  Keys available: {list(results[attack_type].keys())}")

    # Count how many splits we have
    split_count = sum(1 for k in results[attack_type].keys() if k.endswith('_X_train'))
    print(f"  Number of splits found: {split_count}")

    # Show sample sizes for each split
    for prefix in ['heuristic', 'variance', 'pca']:
        train_key = f'{prefix}_X_train'
        if train_key in results[attack_type]:
            X_train = results[attack_type][train_key]
            y_train = results[attack_type].get(f'{prefix}_y_train', None)
            anomalies = int(y_train.sum()) if y_train is not None else 'N/A'
            print(f"  {prefix}: {X_train.shape} samples, {anomalies} anomalies")

#Isolation forest Model

In [ ]:
from sklearn.ensemble import IsolationForest
import numpy as np

print("Training Isolation Forest models with PROPER contamination settings...")

for attack_type, methods in results.items():
    print(f"\n{'='*60}")
    print(f"Processing {attack_type}...")
    print('='*60)

    for selection_method_key in list(methods.keys()):
        if selection_method_key.endswith('_X_train'):
            # Extract data
            X_train = methods[selection_method_key]
            X_val = methods[selection_method_key.replace('_X_train', '_X_val')]
            X_test = methods[selection_method_key.replace('_X_train', '_X_test')]
            y_train = methods[selection_method_key.replace('_X_train', '_y_train')]

            # Get prefix (heuristic/variance/pca)
            prefix = selection_method_key.replace('_X_train', '')

            # Calculate ACTUAL anomaly rate in training data
            train_anomaly_rate = np.mean(y_train == 1)

            # Set contamination based on ACTUAL rate
            # For Isolation Forest, contamination should be slightly higher than actual rate
            # because it's estimating the proportion of outliers
            contamination = min(0.1, max(0.001, train_anomaly_rate * 1.5))

            print(f"  {prefix}: {X_train.shape[0]:,} train samples, {int(y_train.sum())} anomalies ({train_anomaly_rate:.4%})")
            print(f"    Using contamination: {contamination:.4f}")

            # Initialize Isolation Forest with PROPER settings
            isolation_forest_model = IsolationForest(
                n_estimators=100,           # More trees for better stability
                max_samples='auto',         # Use all samples for each tree
                contamination=contamination, # Match actual anomaly rate
                max_features=1.0,           # Use all features
                bootstrap=False,            # Don't bootstrap for anomaly detection
                random_state=42,
                n_jobs=-1,                  # Use all cores
                verbose=0
            )

            # Train the model
            isolation_forest_model.fit(X_train)

            # Calculate anomaly scores for ALL splits
            train_scores = isolation_forest_model.score_samples(X_train)
            val_scores = isolation_forest_model.score_samples(X_val)
            test_scores = isolation_forest_model.score_samples(X_test)

            # For IF, LOWER scores = more anomalous
            # Store as negative scores so higher = more anomalous (consistent with other models)
            train_scores_neg = -train_scores
            val_scores_neg = -val_scores
            test_scores_neg = -test_scores

            # Store everything
            model_key = f'{prefix}_isolation_forest_model'
            train_scores_key = f'{prefix}_train_scores_if'
            val_scores_key = f'{prefix}_val_scores_if'
            test_scores_key = f'{prefix}_test_scores_if'

            results[attack_type][model_key] = isolation_forest_model
            results[attack_type][train_scores_key] = train_scores_neg
            results[attack_type][val_scores_key] = val_scores_neg
            results[attack_type][test_scores_key] = test_scores_neg

            # Also store the contamination used
            results[attack_type][f'{prefix}_if_contamination'] = contamination

            print(f"    ✓ Trained, scores calculated")

print(f"\n{'='*60}")
print("Isolation Forest training complete!")
print('='*60)

print("\n" + "="*80)
print("DETAILED VERIFICATION of Isolation Forest Models")
print("="*80)

found_models = 0
for attack_type in ['brute_force', 'geolocation', 'port_scan', 'privilege']:
    if attack_type in results:
        print(f"\n{attack_type.upper()}:")
        for prefix in ['heuristic', 'variance', 'pca']:
            model_key = f'{prefix}_isolation_forest_model'
            train_scores_key = f'{prefix}_train_scores_if'

            if model_key in results[attack_type]:
                found_models += 1
                model = results[attack_type][model_key]
                contamination = results[attack_type].get(f'{prefix}_if_contamination', 'N/A')

                # Get sample counts
                y_train = results[attack_type].get(f'{prefix}_y_train')
                train_anomalies = int(y_train.sum()) if y_train is not None else 'N/A'

                print(f"  {prefix}:")
                print(f"    Model type: {type(model).__name__}")
                print(f"    Contamination: {contamination}")
                print(f"    Train samples: {results[attack_type][train_scores_key].shape[0]:,}")
                print(f"    Train anomalies: {train_anomalies}")

                # Check score statistics
                scores = results[attack_type][train_scores_key]
                print(f"    Score range: [{scores.min():.3f}, {scores.max():.3f}]")
                print(f"    Mean score: {scores.mean():.3f}")
                print(f"    Std score: {scores.std():.3f}")

                # Check if model is predicting anything
                if hasattr(model, 'predict'):
                    # Sample predictions
                    X_sample = results[attack_type].get(f'{prefix}_X_train')
                    if X_sample is not None and len(X_sample) > 0:
                        sample_pred = model.predict(X_sample[:10])
                        anomalies_pred = sum(sample_pred == -1)
                        print(f"    Sample predictions: {anomalies_pred}/10 flagged as anomalies")
            else:
                print(f"  {prefix}: NO MODEL FOUND (key: {model_key})")
    else:
        print(f"\n{attack_type.upper()}: NOT IN RESULTS DICTIONARY")

print(f"\n{'='*80}")
print(f"SUMMARY: Found {found_models} Isolation Forest models")
print('='*80)

# Quick check for common issues
print("\n" + "="*80)
print("COMMON ISSUES CHECKLIST")
print("="*80)

issues = []
for attack_type in results.keys():
    # Check if splits exist
    for prefix in ['heuristic', 'variance', 'pca']:
        train_key = f'{prefix}_X_train'
        if train_key in results[attack_type]:
            X_train = results[attack_type][train_key]
            y_train = results[attack_type].get(f'{prefix}_y_train')

            if y_train is not None:
                anomaly_rate = np.mean(y_train == 1) if len(y_train) > 0 else 0
                print(f"{attack_type} - {prefix}:")
                print(f"  Samples: {X_train.shape[0]:,}")
                print(f"  Anomalies: {int(y_train.sum()):,}")
                print(f"  Anomaly rate: {anomaly_rate:.4%}")

                if anomaly_rate == 0:
                    issues.append(f"{attack_type}-{prefix}: NO ANOMALIES in training!")
                elif anomaly_rate < 0.001:
                    issues.append(f"{attack_type}-{prefix}: VERY FEW anomalies ({anomaly_rate:.4%})")

if issues:
    print(f"\n⚠️  ISSUES FOUND:")
    for issue in issues:
        print(f"  - {issue}")
else:
    print("\n✅ No major issues found in training data")

#Autoencoders Model training

In [ ]:


# Check GPU availability and configure TensorFlow for better performance
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

# Configure TensorFlow for better performance
tf.config.optimizer.set_jit(True)  # Enable XLA compilation

# Pre-compile a model template to avoid recompiling in loop
def create_autoencoder(input_dim):
    encoding_dim = max(1, input_dim // 4)  # Smaller encoding for faster training

    autoencoder = Sequential([
        Dense(encoding_dim, activation='relu', input_shape=(input_dim,)),
        Dense(input_dim, activation='linear')
    ])

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Collect all training data first, then train in batch
training_tasks = []

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()):
        if selection_method_key.endswith('_X_train'):
            training_tasks.append({
                'attack_type': attack_type,
                'selection_method': selection_method_key,
                'X_train': methods[selection_method_key],
                'X_val': methods[selection_method_key.replace('_X_train', '_X_val')]
            })

print(f"Found {len(training_tasks)} models to train")

# Train models sequentially with progress tracking
for i, task in enumerate(training_tasks):
    print(f"Training model {i+1}/{len(training_tasks)}: {task['attack_type']} - {task['selection_method']}")

    X_train = task['X_train']
    X_val = task['X_val']

    # Skip if data is too small
    if X_train.shape[0] < 10 or X_val.shape[0] < 5:
        print(f"  Skipping - insufficient data: train={X_train.shape[0]}, val={X_val.shape[0]}")
        continue

    input_dim = X_train.shape[1]

    # Create and train model
    autoencoder = create_autoencoder(input_dim)

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,  # Reduced patience for faster training
        restore_best_weights=True,
        verbose=0
    )

    # Train with smaller epochs and larger batch size
    history = autoencoder.fit(
        X_train, X_train,
        epochs=30,  # Reduced epochs
        batch_size=min(64, X_train.shape[0]),  # Adaptive batch size
        shuffle=True,
        validation_data=(X_val, X_val),
        callbacks=[early_stopping],
        verbose=0
    )

    # Calculate reconstruction errors with batching
    train_predictions = autoencoder.predict(X_train, batch_size=256, verbose=0)
    val_predictions = autoencoder.predict(X_val, batch_size=256, verbose=0)

    train_reconstruction_errors = np.mean(np.square(X_train - train_predictions), axis=1)
    val_reconstruction_errors = np.mean(np.square(X_val - val_predictions), axis=1)

    # Store results
    model_key = task['selection_method'].replace('_X_train', '_autoencoder_model')
    train_errors_key = task['selection_method'].replace('_X_train', '_train_reconstruction_errors')
    val_errors_key = task['selection_method'].replace('_X_train', '_val_reconstruction_errors')

    results[task['attack_type']][model_key] = autoencoder
    results[task['attack_type']][train_errors_key] = train_reconstruction_errors
    results[task['attack_type']][val_errors_key] = val_reconstruction_errors

    print(f"  Completed - Final loss: {history.history['loss'][-1]:.4f}")

print("All models trained successfully!")

#Testing the models

In [ ]:
for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()): # Iterate over a copy of keys
        if selection_method_key.endswith('_isolation_forest_model'):
            # Extract method prefix (e.g., 'heuristic', 'variance', 'pca')
            method_prefix = selection_method_key.replace('_isolation_forest_model', '')

            # Retrieve the trained Isolation Forest model
            isolation_forest_model = methods[selection_method_key]

            # Construct the key for the corresponding X_test data
            X_test_key = f'{method_prefix}_X_test'
            X_test = methods[X_test_key]

            # Calculate anomaly scores for X_test
            test_scores = isolation_forest_model.decision_function(X_test)

            # Store the test scores back into the results dictionary
            test_scores_key = f'{method_prefix}_test_scores'
            results[attack_type][test_scores_key] = test_scores

            print(f"Anomaly scores calculated for {attack_type} - {method_prefix}. Shape: {test_scores.shape}")

# Verify one example to ensure test scores are stored
print("\nBrute-force (heuristic) test scores shape:")
print(f"Test scores shape: {results['brute_force']['heuristic_test_scores'].shape}")

In [ ]:
print("\nEvaluating Autoencoder Models on test data...")
for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()):
        if selection_method_key.endswith('_autoencoder_model'):
            # Extract the selection method name (e.g., 'heuristic', 'variance', 'pca')
            method_prefix = selection_method_key.replace('_autoencoder_model', '')

            # Get the trained Autoencoder model
            autoencoder_model = methods[selection_method_key]

            # Get the corresponding X_test data
            X_test_key = f'{method_prefix}_X_test'
            X_test = methods[X_test_key]

            # Predict reconstruction on the test set
            # Ensure X_test has data before predicting
            if X_test.shape[0] > 0:
                test_predictions = autoencoder_model.predict(X_test, batch_size=256, verbose=0)
                # Calculate reconstruction errors
                test_reconstruction_errors = np.mean(np.square(X_test - test_predictions), axis=1)
            else:
                test_reconstruction_errors = np.array([]) # Empty array if no test data

            # Store the test reconstruction errors in the results dictionary
            test_errors_key = f'{method_prefix}_test_reconstruction_errors'
            results[attack_type][test_errors_key] = test_reconstruction_errors

            print(f"  {attack_type} - {method_prefix} Autoencoder test reconstruction errors shape: {test_reconstruction_errors.shape}")

print("\nAll models evaluated on test data. Scores are stored in the 'results' dictionary.")
print("Accessing Isolation Forest test scores for brute_force, heuristic method: results['brute_force']['heuristic_test_scores']")
print("Accessing Autoencoder test reconstruction errors for port_scan, pca method: results['port_scan']['pca_test_reconstruction_errors']")

In [ ]:
df['is_anomaly'] = df['anomaly_label'].map({'anomaly': 1, 'normal': 0})
print("Created 'is_anomaly' column:")
print(df[['anomaly_label', 'is_anomaly']].head())

In [ ]:
# 2. For user-based features (brute_force, geolocation, privilege)
user_ground_truth_df = df.groupby('username')['is_anomaly'].max().reset_index()
user_ground_truth_df.rename(columns={'is_anomaly': 'ground_truth'}, inplace=True)
print("\nUser-based ground truth labels (user_ground_truth_df):")
print(user_ground_truth_df.head())

# 3. For IP/time-window-based features (port_scan)
# Add a 'time_window' column to the original df DataFrame
df['time_window'] = df['timestamp'].dt.floor('15min')

port_scan_ground_truth_df = df.groupby(['source_ip', 'time_window'])['is_anomaly'].max().reset_index()
port_scan_ground_truth_df.rename(columns={'is_anomaly': 'ground_truth'}, inplace=True)
print("\nIP/time-window-based ground truth labels (port_scan_ground_truth_df):")
print(port_scan_ground_truth_df.head())

In [ ]:

# Helper function to get the base DataFrame for an attack type
def get_base_dataframe(attack_type):
    if attack_type == 'brute_force':
        return user_features
    elif attack_type == 'geolocation':
        return geo_features
    elif attack_type == 'privilege':
        return priv_user_features
    elif attack_type == 'port_scan':
        return port_scan_features
    else:
        raise ValueError(f"Unknown attack type: {attack_type}")

# Iterate through each attack type and feature selection method
for attack_type, methods in results.items():
    print(f"Processing ground truth for {attack_type}...")

    # Get the base DataFrame for the current attack type
    base_df = get_base_dataframe(attack_type)

    if attack_type in ['brute_force', 'geolocation', 'privilege']:
        # Merge with user_ground_truth_df
        merged_df = base_df.merge(user_ground_truth_df, on='username', how='left')
        y_full = merged_df['ground_truth'].values
    elif attack_type == 'port_scan':
        # Merge with port_scan_ground_truth_df
        merged_df = base_df.merge(port_scan_ground_truth_df, on=['source_ip', 'time_window'], how='left')
        y_full = merged_df['ground_truth'].values
    else:
        raise ValueError(f"Unhandled attack type for ground truth merging: {attack_type}")

    # Fill any NaN values in y_full (if any arise from merges) with 0 (normal)
    y_full = np.nan_to_num(y_full, nan=0.0)


    sample_X_key = next((k for k in methods if k.endswith('_X_train')), None)
    if sample_X_key is None:
        print(f"  Warning: Could not find any _X_train for {attack_type}, skipping Y splits.")
        continue

    sample_X_train = methods[sample_X_key]

    for selection_method_key in list(methods.keys()):
        if selection_method_key.endswith('_X_train'):
            method_prefix = selection_method_key.replace('_X_train', '')
            X_train = methods[selection_method_key]
            X_val = methods[f'{method_prefix}_X_val']
            X_test = methods[f'{method_prefix}_X_test']


            # First, get the full aligned y for the current feature set. This handles potential filtering/sorting.
            if attack_type in ['brute_force', 'geolocation', 'privilege']:
                current_feature_df = get_base_dataframe(attack_type)
                full_feature_and_y_df = current_feature_df.merge(user_ground_truth_df, on='username', how='left')


            # Align y_full to the *original* (pre-split) feature set it corresponds to (e.g., user_features)
            if attack_type == 'brute_force':
                y_full_aligned = user_features.merge(user_ground_truth_df, on='username', how='left')['ground_truth'].values
            elif attack_type == 'geolocation':
                y_full_aligned = geo_features.merge(user_ground_truth_df, on='username', how='left')['ground_truth'].values
            elif attack_type == 'privilege':
                y_full_aligned = priv_user_features.merge(user_ground_truth_df, on='username', how='left')['ground_truth'].values
            elif attack_type == 'port_scan':
                y_full_aligned = port_scan_features.merge(port_scan_ground_truth_df, on=['source_ip', 'time_window'], how='left')['ground_truth'].values
            y_full_aligned = np.nan_to_num(y_full_aligned, nan=0.0)


            # Use the exact same split parameters as for X to ensure alignment
            y_temp_train, y_test = train_test_split(y_full_aligned, test_size=0.2, random_state=42)
            y_train, y_val = train_test_split(y_temp_train, test_size=0.25, random_state=42)

            # Store the y splits
            results[attack_type][f'{method_prefix}_y_train'] = y_train
            results[attack_type][f'{method_prefix}_y_val'] = y_val
            results[attack_type][f'{method_prefix}_y_test'] = y_test

            print(f"  {attack_type} - {method_prefix}: y_train shape={y_train.shape}, y_val shape={y_val.shape}, y_test shape={y_test.shape}")


print("\nGround truth labels splitting complete.")

# Verify one example
print("\nBrute-force (heuristic) ground truth splits shapes:")
print(f"y_train shape: {results['brute_force']['heuristic_y_train'].shape}")
print(f"y_val shape: {results['brute_force']['heuristic_y_val'].shape}")
print(f"y_test shape: {results['brute_force']['heuristic_y_test'].shape}")

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

print("Evaluating Isolation Forest models performance...")

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()): # Iterate over a copy of keys
        # Check if the key corresponds to the test scores of an Isolation Forest model
        if selection_method_key.endswith('_test_scores') and 'isolation_forest' not in selection_method_key: # Ensure it's IF scores, not AE errors
            method_prefix = selection_method_key.replace('_test_scores', '')

            # Retrieve Isolation Forest test scores
            test_scores = methods[selection_method_key]

            # Retrieve the corresponding y_test ground truth labels
            y_test_key = f'{method_prefix}_y_test'
            y_test = methods[y_test_key]

            # Only calculate if there are actual anomalies in y_test (or normal instances for AP)
            if len(np.unique(y_test)) > 1:
                # Calculate ROC AUC score
                roc_auc = roc_auc_score(y_test, test_scores)

                # Calculate Average Precision score
                avg_precision = average_precision_score(y_test, test_scores)

                # Store these scores in the results dictionary
                results[attack_type][f'{method_prefix}_if_roc_auc'] = roc_auc
                results[attack_type][f'{method_prefix}_if_avg_precision'] = avg_precision

                print(f"  {attack_type} - {method_prefix} IF: ROC AUC = {roc_auc:.4f}, Avg Precision = {avg_precision:.4f}")
            else:
                print(f"  {attack_type} - {method_prefix} IF: Skipping metrics calculation due to single class in y_test (all normal or all anomaly).")

print("\nIsolation Forest model performance evaluation complete.")

# Verify one example to ensure metrics are stored, only if they exist
print("\nBrute-force (heuristic) Isolation Forest metrics:")
if 'heuristic_if_roc_auc' in results['brute_force']:
    print(f"ROC AUC: {results['brute_force']['heuristic_if_roc_auc']:.4f}")
    print(f"Average Precision: {results['brute_force']['heuristic_if_avg_precision']:.4f}")
else:
    print("  Metrics for brute_force - heuristic IF were skipped (single class in y_test).")

In [ ]:
import numpy as np

print("Distribution of 'anomaly_label' in original df:")
print(df['anomaly_label'].value_counts())

print("\nDistribution of 'is_anomaly' in original df (after mapping):")
print(df['is_anomaly'].value_counts())

print("\nDistribution of 'ground_truth' in user_ground_truth_df:")
print(user_ground_truth_df['ground_truth'].value_counts())

print("\nDistribution of 'ground_truth' in port_scan_ground_truth_df:")
print(port_scan_ground_truth_df['ground_truth'].value_counts())

In [ ]:
df['is_anomaly'] = df['anomaly_label'].apply(lambda x: 1 if x != 'normal' else 0)
print("Corrected 'is_anomaly' column creation:")
print(df[['anomaly_label', 'is_anomaly']].head())
print(df['is_anomaly'].value_counts())

user_ground_truth_df = df.groupby('username')['is_anomaly'].max().reset_index()
user_ground_truth_df.rename(columns={'is_anomaly': 'ground_truth'}, inplace=True)
print("\nRe-generated User-based ground truth labels (user_ground_truth_df):")
print(user_ground_truth_df.head())
print(user_ground_truth_df['ground_truth'].value_counts())


df['time_window'] = df['timestamp'].dt.floor('15min')

port_scan_ground_truth_df = df.groupby(['source_ip', 'time_window'])['is_anomaly'].max().reset_index()
port_scan_ground_truth_df.rename(columns={'is_anomaly': 'ground_truth'}, inplace=True)
print("\nRe-generated IP/time-window-based ground truth labels (port_scan_ground_truth_df):")
print(port_scan_ground_truth_df.head())
print(port_scan_ground_truth_df['ground_truth'].value_counts())

In [ ]:

# Helper function to get the base DataFrame for an attack type
def get_base_dataframe(attack_type):
    if attack_type == 'brute_force':
        return user_features
    elif attack_type == 'geolocation':
        return geo_features
    elif attack_type == 'privilege':
        return priv_user_features
    elif attack_type == 'port_scan':
        return port_scan_features
    else:
        raise ValueError(f"Unknown attack type: {attack_type}")

# Iterate through each attack type and feature selection method
for attack_type, methods in results.items():
    print(f"Processing ground truth for {attack_type}...")

    # Align y_full to the *original* (pre-split) feature set it corresponds to (e.g., user_features)
    if attack_type == 'brute_force':
        y_full_aligned = user_features.merge(user_ground_truth_df, on='username', how='left')['ground_truth'].values
    elif attack_type == 'geolocation':
        y_full_aligned = geo_features.merge(user_ground_truth_df, on='username', how='left')['ground_truth'].values
    elif attack_type == 'privilege':
        y_full_aligned = priv_user_features.merge(user_ground_truth_df, on='username', how='left')['ground_truth'].values
    elif attack_type == 'port_scan':
        y_full_aligned = port_scan_features.merge(port_scan_ground_truth_df, on=['source_ip', 'time_window'], how='left')['ground_truth'].values
    y_full_aligned = np.nan_to_num(y_full_aligned, nan=0.0)

    # Find a selection method's X_train to get its size, to ensure consistent splitting
    sample_X_key = next((k for k in methods if k.endswith('_X_train')), None)
    if sample_X_key is None:
        print(f"  Warning: Could not find any _X_train for {attack_type}, skipping Y splits.")
        continue

    # Use the exact same split parameters as for X to ensure alignment
    y_temp_train, y_test = train_test_split(y_full_aligned, test_size=0.2, random_state=42)
    y_train, y_val = train_test_split(y_temp_train, test_size=0.25, random_state=42)

    for selection_method_key in list(methods.keys()):
        if selection_method_key.endswith('_X_train'):
            method_prefix = selection_method_key.replace('_X_train', '')

            # Store the y splits
            results[attack_type][f'{method_prefix}_y_train'] = y_train
            results[attack_type][f'{method_prefix}_y_val'] = y_val
            results[attack_type][f'{method_prefix}_y_test'] = y_test

            print(f"  {attack_type} - {method_prefix}: y_train shape={{y_train.shape}}, y_val shape={{y_val.shape}}, y_test shape={{y_test.shape}}")


print("\nGround truth labels splitting complete.")

# Verify one example
print("\nBrute-force (heuristic) ground truth splits shapes:")
print(f"y_train shape: {results['brute_force']['heuristic_y_train'].shape}")
print(f"y_val shape: {results['brute_force']['heuristic_y_val'].shape}")
print(f"y_test shape: {results['brute_force']['heuristic_y_test'].shape}")

In [ ]:


print("Evaluating Isolation Forest models performance...")

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()): # Iterate over a copy of keys

        if selection_method_key.endswith('_test_scores') and 'if' in selection_method_key:
            method_prefix = selection_method_key.replace('_test_scores', '')


            if_test_scores_key = f'{method_prefix}' # This is incorrect, it should be the actual key with scores
            if_test_scores = methods[selection_method_key]

            # Retrieve the corresponding y_test ground truth labels
            y_test_key = f'{method_prefix.replace("_if", "")}_y_test'
            y_test = methods[y_test_key]

            # Only calculate if there are actual anomalies in y_test (or normal instances for AP)
            if len(np.unique(y_test)) > 1:
                # Calculate ROC AUC score
                roc_auc = roc_auc_score(y_test, if_test_scores)

                # Calculate Average Precision score
                avg_precision = average_precision_score(y_test, if_test_scores)

                # Store these scores in the results dictionary
                results[attack_type][f'{method_prefix}_roc_auc'] = roc_auc
                results[attack_type][f'{method_prefix}_avg_precision'] = avg_precision

                print(f"  {attack_type} - {method_prefix} IF: ROC AUC = {roc_auc:.4f}, Avg Precision = {avg_precision:.4f}")
            else:
                print(f"  {attack_type} - {method_prefix} IF: Skipping metrics calculation due to single class in y_test (all normal or all anomaly).")

print("\nIsolation Forest model performance evaluation complete.")

# Verify one example to ensure metrics are stored, only if they exist
print("\nBrute-force (heuristic) Isolation Forest metrics:")
if 'heuristic_if_test_scores_roc_auc' in results['brute_force']:
    print(f"ROC AUC: {results['brute_force']['heuristic_if_test_scores_roc_auc']:.4f}")
    print(f"Average Precision: {results['brute_force']['heuristic_if_test_scores_avg_precision']:.4f}")
else:

    if 'heuristic_test_scores_roc_auc' in results['brute_force']:
        print(f"ROC AUC: {results['brute_force']['heuristic_test_scores_roc_auc']:.4f}")
        print(f"Average Precision: {results['brute_force']['heuristic_test_scores_avg_precision']:.4f}")
    else:
        print("  Metrics for brute_force - heuristic IF were skipped (single class in y_test) or keys not found.")


In [ ]:
import numpy as np

print("Verifying unique classes in y_test arrays:")
for attack_type, methods in results.items():
    print(f"\nAttack Type: {attack_type}")
    for selection_method_key in list(methods.keys()):
        if selection_method_key.endswith('_y_test'):
            method_prefix = selection_method_key.replace('_y_test', '')
            y_test = methods[selection_method_key]
            unique_classes = np.unique(y_test)
            class_counts = {cls: np.sum(y_test == cls) for cls in unique_classes}
            print(f"  {method_prefix}_y_test: Unique classes = {unique_classes}, Counts = {class_counts}")


In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

print("Evaluating Isolation Forest models performance with corrected ground truth...")

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()):
        # Identify the Isolation Forest test scores keys (e.g., 'heuristic_test_scores', 'variance_test_scores')
        if selection_method_key.endswith('_test_scores') and 'isolation_forest_model' not in selection_method_key:
            method_prefix = selection_method_key.replace('_test_scores', '')

            # Retrieve Isolation Forest test scores
            test_scores = methods[selection_method_key]

            # Retrieve the corresponding y_test ground truth labels
            y_test_key = f'{method_prefix}_y_test'
            y_test = methods[y_test_key]

            # Since the previous step verified that all y_test arrays now contain both classes,
            # this check should now pass for all relevant cases.
            if len(np.unique(y_test)) > 1:
                # Calculate ROC AUC score
                roc_auc = roc_auc_score(y_test, test_scores)

                # Calculate Average Precision score
                avg_precision = average_precision_score(y_test, test_scores)

                # Store these scores in the results dictionary under clear keys
                results[attack_type][f'{method_prefix}_if_roc_auc'] = roc_auc
                results[attack_type][f'{method_prefix}_if_avg_precision'] = avg_precision

                print(f"  {attack_type} - {method_prefix} IF: ROC AUC = {roc_auc:.4f}, Avg Precision = {avg_precision:.4f}")
            else:
                print(f"  {attack_type} - {method_prefix} IF: Still skipping metrics calculation (unexpected single class in y_test).")

print("\nIsolation Forest model performance evaluation complete.")

# Verify one example to ensure metrics are stored, now expecting them to exist
print("\nBrute-force (heuristic) Isolation Forest metrics (re-evaluated):")
if 'heuristic_if_roc_auc' in results['brute_force']:
    print(f"ROC AUC: {results['brute_force']['heuristic_if_roc_auc']:.4f}")
    print(f"Average Precision: {results['brute_force']['heuristic_if_avg_precision']:.4f}")
else:
    print("  Metrics for brute_force - heuristic IF were still not calculated, please investigate further.")


In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

print("Evaluating Autoencoder models performance...")

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()):
        # Identify the Autoencoder test reconstruction errors keys
        if selection_method_key.endswith('_test_reconstruction_errors') and 'autoencoder' not in selection_method_key:
            method_prefix = selection_method_key.replace('_test_reconstruction_errors', '')

            # Retrieve Autoencoder test reconstruction errors
            test_reconstruction_errors = methods[selection_method_key]

            # Retrieve the corresponding y_test ground truth labels
            y_test_key = f'{method_prefix}_y_test'
            y_test = methods[y_test_key]

            # Only calculate if there are actual anomalies in y_test (or normal instances for AP)
            if len(np.unique(y_test)) > 1:
                # Calculate ROC AUC score
                roc_auc = roc_auc_score(y_test, test_reconstruction_errors)

                # Calculate Average Precision score
                avg_precision = average_precision_score(y_test, test_reconstruction_errors)

                # Store these scores in the results dictionary under clear keys
                results[attack_type][f'{method_prefix}_ae_roc_auc'] = roc_auc
                results[attack_type][f'{method_prefix}_ae_avg_precision'] = avg_precision

                print(f"  {attack_type} - {method_prefix} AE: ROC AUC = {roc_auc:.4f}, Avg Precision = {avg_precision:.4f}")
            else:
                print(f"  {attack_type} - {method_prefix} AE: Skipping metrics calculation due to single class in y_test (all normal or all anomaly).")

print("\nAutoencoder model performance evaluation complete.")

# Verify one example to ensure metrics are stored
print("\nBrute-force (heuristic) Autoencoder metrics:")
if 'heuristic_ae_roc_auc' in results['brute_force']:
    print(f"ROC AUC: {results['brute_force']['heuristic_ae_roc_auc']:.4f}")
    print(f"Average Precision: {results['brute_force']['heuristic_ae_avg_precision']:.4f}")
else:
    print("  Metrics for brute_force - heuristic AE were skipped (single class in y_test).")

In [ ]:
print("\n--- Performance Metrics Summary ---")

for attack_type, methods in results.items():
    print(f"\nAttack Type: {attack_type.replace('_', ' ').title()}")
    print("--------------------------------")

    # Iterate through each selection method (heuristic, variance, pca)
    # We'll use a fixed list of prefixes to ensure consistent iteration order
    selection_method_prefixes = ['heuristic', 'variance', 'pca']

    for prefix in selection_method_prefixes:
        print(f"  Feature Selection Method: {prefix.title()}")

        # Isolation Forest Metrics
        if_roc_auc_key = f'{prefix}_if_roc_auc'
        if_avg_precision_key = f'{prefix}_if_avg_precision'

        if if_roc_auc_key in methods and if_avg_precision_key in methods:
            print(f"    Isolation Forest - ROC AUC: {methods[if_roc_auc_key]:.4f}, Avg Precision: {methods[if_avg_precision_key]:.4f}")
        else:
            print(f"    Isolation Forest - Metrics N/A (likely skipped due to single class in y_test).")

        # Autoencoder Metrics
        ae_roc_auc_key = f'{prefix}_ae_roc_auc'
        ae_avg_precision_key = f'{prefix}_ae_avg_precision'

        if ae_roc_auc_key in methods and ae_avg_precision_key in methods:
            print(f"    Autoencoder      - ROC AUC: {methods[ae_roc_auc_key]:.4f}, Avg Precision: {methods[ae_avg_precision_key]:.4f}")
        else:
            print(f"    Autoencoder      - Metrics N/A (likely skipped due to single class in y_test).")

print("\n--- Summary Complete ---")

#saving the models

In [ ]:
joblib.dump(isolation_forest_model, f'isolation_forest_{attack_type}_{method_prefix}.pkl')
joblib.dump(scaler, f'scaler_{attack_type}_{method_prefix}.pkl')

# For Keras autoencoder:
autoencoder.save(f'autoencoder_{attack_type}_{method_prefix}.h5')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Retrieve the Isolation Forest test scores and corresponding ground truth labels
attack_type = 'brute_force'
selection_method = 'pca'

if_test_scores = results[attack_type][f'{selection_method}_test_scores']
y_test = results[attack_type][f'{selection_method}_y_test']

# Create a DataFrame for plotting
plot_df = pd.DataFrame({
    'Anomaly Score': if_test_scores,
    'Ground Truth': y_test
})

# Map ground truth labels to more descriptive names for the legend
plot_df['Ground Truth Label'] = plot_df['Ground Truth'].map({0: 'Normal', 1: 'Anomaly'})

# 2. Create a histogram or a kernel density estimate (KDE) plot
plt.figure(figsize=(10, 6))
sns.histplot(data=plot_df, x='Anomaly Score', hue='Ground Truth Label', kde=True, bins=50, palette='viridis', common_norm=False)

# 4. Add appropriate labels and a title
plt.xlabel('Isolation Forest Anomaly Score')
plt.ylabel('Density / Count')
plt.title(f'Isolation Forest Anomaly Score Distribution for {attack_type.replace("_", " ").title()} ({selection_method.upper()})')
plt.legend(title='Ground Truth')
plt.grid(True, linestyle='--', alpha=0.7)

# 5. Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Retrieve the Isolation Forest test scores and corresponding ground truth labels
attack_type = 'brute_force'
selection_method = 'pca'

if_test_scores = results[attack_type][f'{selection_method}_test_scores']
y_test = results[attack_type][f'{selection_method}_y_test']

# Create a DataFrame for plotting
plot_df = pd.DataFrame({
    'Anomaly Score': if_test_scores,
    'Ground Truth': y_test
})

# Map ground truth labels to more descriptive names for the legend
plot_df['Ground Truth Label'] = plot_df['Ground Truth'].map({0: 'Normal', 1: 'Anomaly'})

# 2. Create a histogram or a kernel density estimate (KDE) plot
plt.figure(figsize=(10, 6))
sns.histplot(data=plot_df, x='Anomaly Score', hue='Ground Truth Label', kde=True, bins=50, palette='viridis', common_norm=False)

# 4. Add appropriate labels and a title
plt.xlabel('Isolation Forest Anomaly Score')
plt.ylabel('Density / Count')
plt.title(f'Isolation Forest Anomaly Score Distribution for {attack_type.replace("_", " ").title()} ({selection_method.upper()})')
plt.grid(True, linestyle='--', alpha=0.7)

# 5. Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# 1. Retrieve the Autoencoder test reconstruction errors and corresponding ground truth labels
attack_type = 'brute_force'
selection_method = 'pca'

ae_test_errors = results[attack_type][f'{selection_method}_test_reconstruction_errors']
y_test = results[attack_type][f'{selection_method}_y_test']

# 2. Create a Pandas DataFrame for plotting
plot_df = pd.DataFrame({
    'Reconstruction Error': ae_test_errors,
    'Ground Truth': y_test
})

# 3. Add a new column to plot_df called 'Ground Truth Label'
plot_df['Ground Truth Label'] = plot_df['Ground Truth'].map({0: 'Normal', 1: 'Anomaly'})

# 4. Create a histogram or KDE plot
plt.figure(figsize=(10, 6))
sns.histplot(data=plot_df, x='Reconstruction Error', hue='Ground Truth Label', kde=True, bins=50, palette='viridis', common_norm=False)

# 5. Add appropriate labels for the x and y axes
plt.xlabel('Autoencoder Reconstruction Error')
plt.ylabel('Density / Count')

# 6. Add a title to the plot
plt.title(f'Autoencoder Reconstruction Error Distribution for {attack_type.replace("_", " ").title()} ({selection_method.upper()})')

# 7. Add a grid to the plot for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# 8. Display the plot
plt.show()

## Compare Model Performance (ROC AUC)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Initialize an empty list to store performance data
performance_data = []

# Define selection method prefixes for iteration consistency
selection_method_prefixes = ['heuristic', 'variance', 'pca']

# 2. Iterate through each attack_type and its methods in the results dictionary
for attack_type, methods in results.items():
    # 3. For each attack_type, iterate through the selection_method_prefixes
    for prefix in selection_method_prefixes:
        # 4. Check for Isolation Forest ROC AUC scores
        if_roc_auc_key = f'{prefix}_if_roc_auc'
        if if_roc_auc_key in methods:
            performance_data.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Isolation Forest',
                'Metric': 'ROC AUC',
                'Score': methods[if_roc_auc_key]
            })

        # 5. Check for Autoencoder ROC AUC scores
        ae_roc_auc_key = f'{prefix}_ae_roc_auc'
        if ae_roc_auc_key in methods:
            performance_data.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Autoencoder',
                'Metric': 'ROC AUC',
                'Score': methods[ae_roc_auc_key]
            })

# 6. Convert the list of performance data into a Pandas DataFrame
performance_df = pd.DataFrame(performance_data)

# 7. Filter the DataFrame to include only 'ROC AUC' metrics (redundant here, but good practice)
roc_auc_df = performance_df[performance_df['Metric'] == 'ROC AUC']

# 8. Create a grouped bar chart using seaborn.catplot
g = sns.catplot(data=roc_auc_df, x='Attack Type', y='Score', hue='Model', col='Feature Selection',
                kind='bar', palette='Dark2', dodge=True, col_wrap=3, height=5, aspect=1.2)

# 9. Add appropriate labels and a title
g.set_axis_labels('Attack Type', 'ROC AUC Score')
g.set_titles('Feature Selection: {col_name}')
g.set_xticklabels(rotation=45, ha='right')
plt.suptitle('ROC AUC Score Comparison Across Attack Types, Feature Selection, and Models', y=1.02, fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap

# Display the plot
plt.show()


## Compare Model Performance (Average Precision)

### Subtask:
Create a bar chart to compare the Average Precision scores across all attack types, feature selection methods, and models (Isolation Forest vs. Autoencoder). Ensure the plot is clearly labeled with legends to distinguish between models and methods.


**Reasoning**:
I will extract the Average Precision scores from the `results` dictionary, organize them into a Pandas DataFrame, and then generate a faceted bar chart using `seaborn.catplot` to visualize the comparison across different attack types, feature selection methods, and models as per the instructions.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Initialize an empty list to store Average Precision scores
performance_data_ap = []

# 2. Define selection method prefixes for iteration consistency
selection_method_prefixes = ['heuristic', 'variance', 'pca']

# 3. Iterate through each attack_type and its methods in the results dictionary
for attack_type, methods in results.items():
    # a. For each attack_type, iterate through the selection_method_prefixes
    for prefix in selection_method_prefixes:
        # i. Check for Isolation Forest Average Precision scores
        if_avg_precision_key = f'{prefix}_if_avg_precision'
        if if_avg_precision_key in methods:
            performance_data_ap.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Isolation Forest',
                'Metric': 'Average Precision',
                'Score': methods[if_avg_precision_key]
            })

        # ii. Check for Autoencoder Average Precision scores
        ae_avg_precision_key = f'{prefix}_ae_avg_precision'
        if ae_avg_precision_key in methods:
            performance_data_ap.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Autoencoder',
                'Metric': 'Average Precision',
                'Score': methods[ae_avg_precision_key]
            })

# 4. Convert the list of performance data into a Pandas DataFrame
avg_precision_df = pd.DataFrame(performance_data_ap)

# 5. Create a grouped bar chart using seaborn.catplot
g = sns.catplot(data=avg_precision_df, x='Attack Type', y='Score', hue='Model', col='Feature Selection',
                kind='bar', palette='Dark2', dodge=True, col_wrap=3, height=5, aspect=1.2)

# 6. Set the x-axis label to 'Attack Type' and the y-axis label to 'Average Precision Score'
g.set_axis_labels('Attack Type', 'Average Precision Score')
# 7. Set the titles for each facet
g.set_titles('Feature Selection: {col_name}')
# 8. Rotate the x-tick labels by 45 degrees and align them to the right
g.set_xticklabels(rotation=45, ha='right')
# 9. Add a main title to the plot
plt.suptitle('Average Precision Score Comparison Across Attack Types, Feature Selection, and Models', y=1.02, fontsize=16)
# 10. Add a grid to the y-axis for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)
# 11. Adjust the plot layout to prevent the super title from overlapping
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap

# 12. Display the plot
plt.show()

## Visualize Best Performing Model



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# 1. Initialize variables to store the best ROC AUC score and its details
best_roc_auc = -1
best_attack_type = None
best_selection_method = None
best_model_type = None

# 2. Iterate through the results dictionary to find the combination with the highest ROC AUC score
for attack_type, methods in results.items():
    for prefix in ['heuristic', 'variance', 'pca']:
        # Check Isolation Forest ROC AUC
        if_roc_auc_key = f'{prefix}_if_roc_auc'
        if if_roc_auc_key in methods:
            score = methods[if_roc_auc_key]
            if score > best_roc_auc:
                best_roc_auc = score
                best_attack_type = attack_type
                best_selection_method = prefix
                best_model_type = 'Isolation Forest'

        # Check Autoencoder ROC AUC
        ae_roc_auc_key = f'{prefix}_ae_roc_auc'
        if ae_roc_auc_key in methods:
            score = methods[ae_roc_auc_key]
            if score > best_roc_auc:
                best_roc_auc = score
                best_attack_type = attack_type
                best_selection_method = prefix
                best_model_type = 'Autoencoder'

print(f"Best Model Found: {best_model_type} for {best_attack_type.replace('_', ' ').title()} with {best_selection_method.title()} features (ROC AUC: {best_roc_auc:.4f})")

# 3. Retrieve the test anomaly scores/reconstruction errors and y_test labels for the best combination
if best_model_type == 'Isolation Forest':
    test_scores_key = f'{best_selection_method}_test_scores'
    anomaly_scores = results[best_attack_type][test_scores_key]
    metric_label = 'Anomaly Score'
else: # Autoencoder
    test_errors_key = f'{best_selection_method}_test_reconstruction_errors'
    anomaly_scores = results[best_attack_type][test_errors_key]
    metric_label = 'Reconstruction Error'

y_test_key = f'{best_selection_method}_y_test'
y_test = results[best_attack_type][y_test_key]

# Create a DataFrame for plotting
plot_df = pd.DataFrame({
    metric_label: anomaly_scores,
    'Ground Truth': y_test
})

# Map ground truth labels to more descriptive names for the legend
plot_df['Ground Truth Label'] = plot_df['Ground Truth'].map({0: 'Normal', 1: 'Anomaly'})

# 4. Create a scatter plot
plt.figure(figsize=(12, 7))
sns.scatterplot(
    data=plot_df,
    x=np.random.rand(len(plot_df)) * (plot_df.index.max() - plot_df.index.min()) + plot_df.index.min(), # Jittered x-axis
    y=metric_label,
    hue='Ground Truth Label',
    palette={'Normal': 'skyblue', 'Anomaly': 'red'},
    alpha=0.6,
    s=10 # Smaller point size for large datasets
)

# 5. Add appropriate labels and a title
plt.xlabel('Data Point Index (Jittered)')
plt.ylabel(metric_label)
plt.title(f'{best_model_type} Anomaly Detection for {best_attack_type.replace("_", " ").title()} (Features: {best_selection_method.title()})\nROC AUC: {best_roc_auc:.4f}')
plt.legend(title='Ground Truth')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# Task
The last step successfully created visualizations for Isolation Forest and Autoencoder models, comparing their performance using ROC AUC and Average Precision metrics, and identifying the overall best performing model.

The next steps will focus on enhancing the anomaly detection system by introducing two new models, One-Class SVM (OCSVM) and Local Outlier Factor (LOF), and refining our approach to data imbalance and hyperparameter tuning.

I will now proceed with the following actions:

1.  **Address Data Imbalance (Conceptual)**: I will add a text cell discussing the challenges of data imbalance in anomaly detection, why traditional oversampling methods like SMOTE are unsuitable for high-dimensional anomaly detection, and how model-specific parameters and robust evaluation metrics (like ROC AUC and Average Precision) are more appropriate ways to handle this.
2.  **Implement OCSVM Model Training and Scoring**: I will integrate One-Class SVM models into our pipeline. For each attack type and feature selection method, OCSVM models will be trained on the training data. Anomaly scores will then be generated for the training, validation, and test sets and stored in the `results` dictionary.
3.  **Implement LOF Model Training and Scoring**: Similarly, Local Outlier Factor models will be added. For each attack type and feature selection method, LOF models will be trained, and anomaly scores will be computed for the training, validation, and test sets, also stored in the `results` dictionary.
4.  **Discuss Hyperparameter Tuning**: I will add a text cell to outline the importance of hyperparameter tuning for all models, specifically `contamination` for Isolation Forest and OCSVM, and `n_neighbors` for LOF. I'll explain how validation sets can be used to optimize thresholds and parameters to maximize Average Precision.
5.  **Evaluate New Models**: The existing evaluation logic will be extended to calculate ROC AUC and Average Precision scores for the newly added OCSVM and LOF models across all attack types and feature selection methods, ensuring comprehensive performance comparison.
6.  **Update Performance Visualizations**: I will generate updated grouped bar charts to include OCSVM and LOF, comparing ROC AUC and Average Precision scores across all models, attack types, and feature selection methods.
7.  **Update Best Performing Model Visualization**: The visualization for the best performing model will be updated to include OCSVM and LOF in the comparison, ensuring the overall top model is visualized.
8.  **Final Summary**: Finally, I will provide a summary of the performance of all models, discuss the effectiveness of different approaches, and suggest future steps.

Starting with the conceptual discussion on data imbalance.

## Understand and Address Data Imbalance

### Subtask:
Discuss and implement strategies to address data imbalance suitable for high-dimensional anomaly detection. Explain why traditional oversampling (like SMOTE) is unsuitable and focus on methods such as model-specific parameters (e.g., `contamination` in Isolation Forest/OCSVM), weighted loss functions for Autoencoders (if applicable), and robust evaluation metrics (ROC AUC, Average Precision) as primary means of handling imbalance, rather than modifying the raw dataset.


## Understanding and Addressing Data Imbalance in Anomaly Detection

Data imbalance is a pervasive challenge in anomaly detection, where anomalies (the target class) are by definition rare compared to normal instances. This inherent rarity leads to several difficulties, especially in high-dimensional datasets:

### Challenges of Data Imbalance in High-Dimensional Anomaly Detection:
1.  **Model Bias**: Most machine learning algorithms are designed to maximize overall accuracy. With imbalanced data, they tend to become biased towards the majority (normal) class, often classifying anomalous instances as normal, simply because there are so few of them.
2.  **Poor Anomaly Detection**: Models might achieve high accuracy simply by predicting the majority class for all instances, rendering them useless for identifying the rare anomalies we are interested in.
3.  **Increased Noise in High Dimensions**: In high-dimensional spaces, the data becomes sparse, and the concept of 'distance' can become less meaningful. Imbalance exacerbates this, as the few anomalous samples are widely scattered, making it harder for models to learn their distinct patterns.
4.  **Computational Cost**: Modifying high-dimensional datasets can be computationally intensive and might lead to the curse of dimensionality issues during data generation or transformation.

### Why Traditional Oversampling (e.g., SMOTE) is Unsuitable:
Traditional oversampling techniques like SMOTE (Synthetic Minority Oversampling Technique) work by creating synthetic samples of the minority class. While effective for classification tasks with balanced classes, they are generally **unsuitable for anomaly detection** for several critical reasons:
1.  **Definition of Anomaly**: Anomalies are rare, distinct, and often extreme observations. Generating synthetic anomalies by interpolating between existing anomalies fundamentally contradicts this definition. It assumes that anomalies exist on a manifold that can be linearly (or non-linearly, with variants) interpolated, which is often not true. Anomalies are unique, not 'average' versions of other anomalies.
2.  **Risk of Smearing Anomaly Boundaries**: Synthetic anomaly generation can create 'normal-looking' anomalies or 'anomaly-looking' normal points, effectively smearing the boundary between normal and anomalous regions. This can make it harder for the model to distinguish true anomalies.
3.  **Introduction of Noise**: In high-dimensional spaces, SMOTE can easily generate synthetic samples that are noisy or not representative of actual anomalies, further obscuring the true anomaly patterns.
4.  **No New Information**: Synthetic samples don't introduce new information about the underlying data distribution; they merely replicate existing patterns (or variations thereof). True anomalies are, by nature, novel.

### Alternative Strategies for Addressing Data Imbalance:
Instead of modifying the dataset itself, the focus in anomaly detection shifts to using techniques that are inherently designed to handle imbalance or are robust to it. These include:

1.  **Model-Specific Parameters:**
    *   **`contamination` Parameter (Isolation Forest, One-Class SVM):** Algorithms like Isolation Forest and One-Class SVM (OCSVM) often include a `contamination` parameter. This parameter explicitly tells the model the expected proportion of anomalies in the dataset. By setting this parameter, the model can adjust its decision boundary to identify a certain percentage of the most extreme observations as anomalies. This directly addresses the imbalance by acknowledging the rarity of anomalies during model training and prediction.
    *   **Thresholding**: Many anomaly detection models output an anomaly score. Instead of a fixed classification, a threshold can be set on these scores to determine what constitutes an anomaly. This threshold can be chosen based on domain knowledge or desired recall/precision trade-offs, regardless of the inherent class imbalance.

2.  **Weighted Loss Functions (for Autoencoders/Neural Networks):**
    *   For reconstruction-based models like Autoencoders, if you were to train them in a supervised manner (e.g., for classification between normal/anomaly), you could use weighted loss functions. This involves assigning a higher penalty to misclassifications of the minority (anomaly) class during training. However, autoencoders are primarily unsupervised for anomaly detection, learning to reconstruct normal data, and anomalies result in high reconstruction errors. The 'imbalance' is handled by the unsupervised nature of learning *only* the normal class.

3.  **Robust Evaluation Metrics:**
    Traditional accuracy is a misleading metric for imbalanced datasets. Instead, robust metrics that focus on the performance of the minority class are crucial:
    *   **ROC AUC (Receiver Operating Characteristic Area Under the Curve):** This metric evaluates a model's ability to discriminate between classes across various threshold settings. It is less sensitive to class imbalance than accuracy because it considers both true positive rate and false positive rate.
    *   **Average Precision (AP) / PR AUC (Precision-Recall Area Under the Curve):** AP is particularly well-suited for highly imbalanced datasets. It focuses on the model's performance on the positive class (anomalies) and is more sensitive to the trade-off between precision and recall for anomalies. A high AP score indicates that a model is good at finding anomalies without generating too many false positives.
    *   **F1-Score, Recall, Precision**: These metrics can also be useful, but they depend on a specific threshold. Reporting them for the minority class helps understand model performance.

By focusing on these model-centric and evaluation-centric strategies, we can effectively build and assess anomaly detection systems in the presence of severe data imbalance, without resorting to data modifications that could undermine the integrity of anomaly patterns.

## Add OCSVM Model Training and Scoring

### Subtask:
Implement One-Class SVM (OCSVM) models for each attack type and feature selection method. Train the OCSVM models on the training data and calculate anomaly scores for the training, validation, and test sets. These scores will be stored in the 'results' dictionary. Highlight how OCSVM's nature is designed for outlier detection in imbalanced datasets.


**Reasoning**:
The subtask requires implementing One-Class SVM (OCSVM) models. First, I will import the necessary `OneClassSVM` class from `sklearn.svm`.



In [ ]:
from sklearn.svm import OneClassSVM

print("OneClassSVM imported successfully.")

**Reasoning**:
Now that the `OneClassSVM` class is imported, I will proceed to implement the OCSVM model training and scoring as per the subtask instructions. This involves iterating through the `results` dictionary, initializing an OCSVM model for each attack type and feature selection method, training it on the training data, and then calculating and storing anomaly scores for the training, validation, and test sets.



In [ ]:
print("Training One-Class SVM models...")

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()):
        if selection_method_key.endswith('_X_train'):
            # Extract training, validation, and test data
            X_train = methods[selection_method_key]
            X_val = methods[selection_method_key.replace('_X_train', '_X_val')]
            X_test = methods[selection_method_key.replace('_X_train', '_X_test')]

            # Initialize One-Class SVM model
            # nu is an upper bound on the fraction of training errors and a lower bound of the fraction of support vectors
            # It is analogous to contamination in IsolationForest
            ocsvm_model = OneClassSVM(nu=0.01, kernel='rbf', gamma='auto') # Using 'auto' for gamma to simplify for now

            # Train the model (OCSVM is unsupervised, only uses X_train)
            # Ensure X_train has sufficient samples (at least 2 for OCSVM)
            if X_train.shape[0] > 1:
                ocsvm_model.fit(X_train)

                # Calculate anomaly scores using decision_function
                # Lower values are more anomalous, higher are more normal
                train_scores_ocsvm = ocsvm_model.decision_function(X_train)
                val_scores_ocsvm = ocsvm_model.decision_function(X_val)
                test_scores_ocsvm = ocsvm_model.decision_function(X_test)

                # Store the model and scores back into the results dictionary
                model_key = selection_method_key.replace('_X_train', '_ocsvm_model')
                train_scores_key = selection_method_key.replace('_X_train', '_train_scores_ocsvm')
                val_scores_key = selection_method_key.replace('_X_train', '_val_scores_ocsvm')
                test_scores_key = selection_method_key.replace('_X_train', '_test_scores_ocsvm')

                results[attack_type][model_key] = ocsvm_model
                results[attack_type][train_scores_key] = train_scores_ocsvm
                results[attack_type][val_scores_key] = val_scores_ocsvm
                results[attack_type][test_scores_key] = test_scores_ocsvm

                print(f"  Trained OCSVM for {attack_type} - {selection_method_key.replace('_X_train', '')}")
            else:
                print(f"  Skipping OCSVM for {attack_type} - {selection_method_key.replace('_X_train', '')}: Insufficient training data ({X_train.shape[0]} samples).")


# Verify one example to ensure models and scores are stored
print("\nVerification: Brute-force (heuristic) OCSVM model and scores shapes:")
if 'brute_force' in results and 'heuristic_ocsvm_model' in results['brute_force']:
    print(f"Model type: {type(results['brute_force']['heuristic_ocsvm_model'])}")
    print(f"Train scores shape: {results['brute_force']['heuristic_train_scores_ocsvm'].shape}")
    print(f"Validation scores shape: {results['brute_force']['heuristic_val_scores_ocsvm'].shape}")
    print(f"Test scores shape: {results['brute_force']['heuristic_test_scores_ocsvm'].shape}")
else:
    print("  OCSVM model for brute_force - heuristic not found or skipped.")

print("\nOne-Class SVM model training and scoring complete.")

**Reasoning**:
I will now implement the Local Outlier Factor (LOF) models. This involves importing the `LocalOutlierFactor` class, iterating through the `results` dictionary, and for each attack type and feature selection method, initializing, training, and scoring an LOF model. The model and its scores will then be stored in the `results` dictionary.



In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
import numpy as np
import time

print("Training Local Outlier Factor models (OPTIMIZED)...")
start_time = time.time()

# GLOBAL OPTIMIZATIONS
SCALING_ENABLED = True  # Scaling can help LOF convergence
MAX_SAMPLES = 3000      # Maximum samples for training
N_NEIGHBORS = 15        # Fixed for consistency
BATCH_SIZE = 2000       # For scoring

lof_cache = {}

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()):
        if not selection_method_key.endswith('_X_train'):
            continue

        # Quick check for data existence
        X_train = methods.get(selection_method_key)
        if X_train is None or len(X_train) <= N_NEIGHBORS:
            continue

        # Get corresponding validation and test data
        base_key = selection_method_key.replace('_X_train', '')
        X_val = methods.get(f'{base_key}_X_val')
        X_test = methods.get(f'{base_key}_X_test')

        if X_val is None or X_test is None:
            continue

        print(f"  Processing {attack_type} - {base_key}...", end="")

        try:
            # OPTIMIZATION: Scale data for better LOF performance
            if SCALING_ENABLED:
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_val_scaled = scaler.transform(X_val)
                X_test_scaled = scaler.transform(X_test)
            else:
                X_train_scaled = X_train
                X_val_scaled = X_val
                X_test_scaled = X_test

            # OPTIMIZATION: Use subset for training
            if len(X_train_scaled) > MAX_SAMPLES:
                idx = np.random.RandomState(42).choice(
                    len(X_train_scaled),
                    MAX_SAMPLES,
                    replace=False
                )
                X_train_fit = X_train_scaled[idx]
            else:
                X_train_fit = X_train_scaled

            # Check cache based on data characteristics
            cache_key = f"{X_train_fit.shape}_{hash(tuple(X_train_fit.mean(axis=0).tobytes()))}"

            if cache_key in lof_cache:
                lof_model = lof_cache[cache_key]
                model_trained = False
            else:
                # Train LOF with memory-efficient parameters
                lof_model = LocalOutlierFactor(
                    n_neighbors=N_NEIGHBORS,
                    algorithm='kd_tree',  # Faster for moderate dimensions
                    leaf_size=20,  # Smaller leaf = faster queries
                    metric='minkowski',
                    p=2,
                    contamination='auto',
                    novelty=True,
                    n_jobs=1  # Single job for stability
                )

                # Train
                lof_model.fit(X_train_fit)
                lof_cache[cache_key] = lof_model
                model_trained = True

            # Efficient scoring with progress bar for large datasets
            def score_dataset(X, dataset_name=""):
                if len(X) == 0:
                    return np.array([])

                # Score in batches
                scores = []
                n_batches = max(1, len(X) // BATCH_SIZE)

                for i in range(0, len(X), BATCH_SIZE):
                    batch = X[i:i + BATCH_SIZE]
                    batch_scores = lof_model.decision_function(batch)
                    scores.append(batch_scores)

                return np.concatenate(scores) if scores else np.array([])

            # Get scores
            train_scores = score_dataset(X_train_scaled, "train")
            val_scores = score_dataset(X_val_scaled, "val")
            test_scores = score_dataset(X_test_scaled, "test")

            # Store results
            results[attack_type][f'{base_key}_lof_model'] = lof_model
            results[attack_type][f'{base_key}_train_scores_lof'] = train_scores
            results[attack_type][f'{base_key}_val_scores_lof'] = val_scores
            results[attack_type][f'{base_key}_test_scores_lof'] = test_scores

            # Store scaler if used
            if SCALING_ENABLED:
                results[attack_type][f'{base_key}_lof_scaler'] = scaler

            print(f" ✓ ({'trained' if model_trained else 'cached'})")

        except Exception as e:
            print(f" ✗ Error: {str(e)[:50]}...")
            continue

print(f"\nCompleted in {time.time() - start_time:.2f} seconds")
print(f"Models trained/cached: {len(lof_cache)}")

# Task
## Hyperparameter Tuning and Threshold Optimization

### Subtask:
Discuss the importance of hyperparameter tuning for all models, especially the `contamination` parameter for Isolation Forest and OCSVM, and `n_neighbors` for LOF. Explain how to optimize anomaly detection thresholds, potentially using validation sets, to maximize metrics like Average Precision, which are crucial for imbalanced datasets, rather than relying solely on fixed contamination values.

### Reasoning:
I will add a text cell to outline the importance of hyperparameter tuning for all models, specifically `contamination` for Isolation Forest and OCSVM, and `n_neighbors` for LOF. I'll explain how validation sets can be used to optimize thresholds and parameters to maximize Average Precision. This addresses the hyperparameter tuning and threshold optimization part of the plan.

## Add LOF Model Training and Scoring

### Subtask:
Implement Local Outlier Factor (LOF) models for each attack type and feature selection method. Train the LOF models on the training data and calculate anomaly scores for the training, validation, and test sets. These scores will be stored in the 'results' dictionary. Emphasize LOF's density-based approach, which naturally adapts to varying densities of normal and anomalous data.


## Hyperparameter Tuning and Threshold Optimization

### Subtask:
Discuss the importance of hyperparameter tuning for all models, especially the `contamination` parameter for Isolation Forest and OCSVM, and `n_neighbors` for LOF. Explain how to optimize anomaly detection thresholds, potentially using validation sets, to maximize metrics like Average Precision, which are crucial for imbalanced datasets, rather than relying solely on fixed contamination values.


## Hyperparameter Tuning and Threshold Optimization

Anomaly detection models, while powerful, often rely heavily on carefully selected hyperparameters to perform optimally. Unlike supervised learning where a clear target class guides optimization, unsupervised anomaly detection models require a nuanced approach to tuning, especially given the inherent imbalance of anomaly datasets.

### Importance of Hyperparameter Tuning

Hyperparameters dictate the learning process and structure of a model, significantly influencing its ability to distinguish between normal and anomalous instances. Incorrectly chosen hyperparameters can lead to models that either miss many anomalies (high false negatives) or flag too many normal instances as anomalous (high false positives).

*   **Isolation Forest:** The `contamination` parameter is crucial. It represents the proportion of outliers in the data. While often set to an estimated value, tuning it allows the model to adjust its internal thresholds and effectively isolate a more accurate number of anomalies. The `n_estimators` (number of trees) and `max_features` (features to sample for each tree) also impact the robustness and diversity of the forest.
*   **One-Class SVM (OCSVM):** Similar to Isolation Forest, `nu` (an upper bound on the fraction of training errors and a lower bound on the fraction of support vectors) is analogous to `contamination`. The `kernel` type (e.g., 'rbf', 'linear') and its associated parameters like `gamma` (for 'rbf' kernel) define the shape of the decision boundary. Tuning `nu` and `gamma` is vital for OCSVM to accurately delineate the normal class boundary.
*   **Local Outlier Factor (LOF):** The `n_neighbors` parameter is central to LOF. It defines the number of neighbors considered to compute the local density of a sample. The choice of `n_neighbors` directly impacts the 'locality' of the outlier definition; a small `n_neighbors` might make the model sensitive to very local fluctuations, while a large `n_neighbors` might smooth out local structures, potentially missing subtle anomalies. The `contamination` parameter in LOF also helps set a threshold for classification.

### Optimizing Anomaly Detection Thresholds using Validation Sets

Many anomaly detection models output an 'anomaly score' rather than a direct binary classification. A threshold must be applied to these scores to classify instances as normal or anomalous. Relying solely on a fixed `contamination` value (e.g., 0.01) might not be optimal, as the true proportion of anomalies can vary, and what constitutes an 'anomaly' might need to be contextually defined based on business needs (e.g., prioritizing recall over precision).

**Process for Threshold Optimization:**
1.  **Generate Scores:** Train models on the training set and generate anomaly scores for the validation set.
2.  **Iterate Thresholds:** Systematically evaluate a range of potential thresholds on the validation set's anomaly scores.
3.  **Evaluate Metrics:** For each threshold, calculate relevant evaluation metrics (e.g., ROC AUC, Average Precision, F1-score).
4.  **Select Best Threshold:** Choose the threshold that maximizes the desired metric on the validation set. For example, if the goal is to identify as many true anomalies as possible without an excessive number of false alarms, a threshold maximizing Average Precision would be preferred.

### Why Average Precision is Crucial for Imbalanced Datasets

In highly imbalanced anomaly detection scenarios, metrics like standard accuracy are misleading. A model predicting everything as 'normal' might achieve 99% accuracy but fail to detect any anomalies. Metrics like ROC AUC are better, as they are less sensitive to class imbalance, evaluating the model's overall discriminative power.

However, **Average Precision (AP) or PR AUC (Precision-Recall Area Under the Curve) is particularly critical** for threshold optimization in imbalanced anomaly detection for the following reasons:

*   **Focus on the Minority Class:** AP focuses directly on the performance of the positive (anomaly) class. It measures the area under the Precision-Recall curve, which plots precision against recall for various thresholds.
*   **Sensitive to Positive Predictive Value:** Precision (`TP / (TP + FP)`) is a crucial measure of how many of the detected anomalies are actual anomalies. In fraud detection or security breach detection, a high precision is often desired to minimize investigation costs associated with false alarms.
*   **Addresses Imbalance Directly:** Unlike ROC AUC, which can look optimistic with severe class imbalance (due to the True Negative rate dominating the False Positive rate), AP provides a more realistic and often lower score, reflecting the true challenge of detecting rare events. A high AP score indicates that the model can find anomalies with high confidence and relatively few false positives.

By optimizing hyperparameters and thresholds against Average Precision on a validation set, we ensure that our models are tuned to effectively identify the rare anomalous events, which is the primary goal of anomaly detection in imbalanced datasets.

## Evaluate New Models

### Subtask:
Extend the existing evaluation logic to calculate ROC AUC and Average Precision scores for the newly added OCSVM and LOF models across all attack types and feature selection methods.


**Reasoning**:
I will calculate ROC AUC and Average Precision scores for the newly added OCSVM and LOF models across all attack types and feature selection methods, and store them in the `results` dictionary.



In [ ]:
print("Evaluating OCSVM and LOF models performance...")

for attack_type, methods in results.items():
    for selection_method_key in list(methods.keys()):
        # Evaluate OCSVM models
        if selection_method_key.endswith('_test_scores_ocsvm'):
            method_prefix = selection_method_key.replace('_test_scores_ocsvm', '')

            ocsvm_test_scores = methods[selection_method_key]
            y_test_key = f'{method_prefix}_y_test'
            y_test = methods[y_test_key]

            if len(np.unique(y_test)) > 1:
                ocsvm_roc_auc = roc_auc_score(y_test, ocsvm_test_scores)
                ocsvm_avg_precision = average_precision_score(y_test, ocsvm_test_scores)

                results[attack_type][f'{method_prefix}_ocsvm_roc_auc'] = ocsvm_roc_auc
                results[attack_type][f'{method_prefix}_ocsvm_avg_precision'] = ocsvm_avg_precision

                print(f"  {attack_type} - {method_prefix} OCSVM: ROC AUC = {ocsvm_roc_auc:.4f}, Avg Precision = {ocsvm_avg_precision:.4f}")
            else:
                print(f"  {attack_type} - {method_prefix} OCSVM: Skipping metrics calculation due to single class in y_test.")

        # Evaluate LOF models
        elif selection_method_key.endswith('_test_scores_lof'):
            method_prefix = selection_method_key.replace('_test_scores_lof', '')

            lof_test_scores = methods[selection_method_key]
            y_test_key = f'{method_prefix}_y_test'
            y_test = methods[y_test_key]

            if len(np.unique(y_test)) > 1:
                lof_roc_auc = roc_auc_score(y_test, lof_test_scores)
                lof_avg_precision = average_precision_score(y_test, lof_test_scores)

                results[attack_type][f'{method_prefix}_lof_roc_auc'] = lof_roc_auc
                results[attack_type][f'{method_prefix}_lof_avg_precision'] = lof_avg_precision

                print(f"  {attack_type} - {method_prefix} LOF: ROC AUC = {lof_roc_auc:.4f}, Avg Precision = {lof_avg_precision:.4f}")
            else:
                print(f"  {attack_type} - {method_prefix} LOF: Skipping metrics calculation due to single class in y_test.")

print("\nOCSVM and LOF model performance evaluation complete.")

# Verify one example to ensure metrics are stored
print("\nBrute-force (heuristic) OCSVM and LOF metrics:")
if 'heuristic_ocsvm_roc_auc' in results['brute_force']:
    print(f"OCSVM ROC AUC: {results['brute_force']['heuristic_ocsvm_roc_auc']:.4f}")
    print(f"OCSVM Average Precision: {results['brute_force']['heuristic_ocsvm_avg_precision']:.4f}")
else:
    print("  OCSVM metrics for brute_force - heuristic were skipped.")

if 'heuristic_lof_roc_auc' in results['brute_force']:
    print(f"LOF ROC AUC: {results['brute_force']['heuristic_lof_roc_auc']:.4f}")
    print(f"LOF Average Precision: {results['brute_force']['heuristic_lof_avg_precision']:.4f}")
else:
    print("  LOF metrics for brute_force - heuristic were skipped.")

In [ ]:
# === PORT_SCAN quick tuning + threshold selection (run AFTER splits & training) ===
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_curve
import numpy as np

atk = 'port_scan'
best_summary = []

for prefix in ['heuristic','variance','pca']:
    X_train = results[atk].get(f'{prefix}_X_train')
    X_val = results[atk].get(f'{prefix}_X_val')
    X_test = results[atk].get(f'{prefix}_X_test')
    y_val = results[atk].get(f'{prefix}_y_val')
    y_test = results[atk].get(f'{prefix}_y_test')
    scaler = results[atk].get(f'{prefix}_scaler')

    if X_train is None or X_val is None or X_test is None:
        print(f"{atk}-{prefix}: missing data, skipping.")
        continue

    # Standardize (fit on train)
    scaler_local = StandardScaler().fit(X_train) if scaler is None else scaler
    Xtr = scaler_local.transform(X_train)
    Xv = scaler_local.transform(X_val)
    Xt = scaler_local.transform(X_test)
    results[atk][f'{prefix}_scaler'] = scaler_local  # save back if absent

    # 1) IsolationForest quick grid
    best_if = {'ap': -1}
    for contam in [0.005, 0.01, 0.02]:
        try:
            m = IsolationForest(contamination=contam, n_estimators=100, max_samples=min(512, len(Xtr)), random_state=42, n_jobs=1)
            m.fit(Xtr)
            val_scores = -m.decision_function(Xv)  # higher = anomalous
            ap = average_precision_score(y_val, val_scores)
            if ap > best_if['ap']:
                best_if = {'ap':ap,'model':m,'contamination':contam,'val_scores':val_scores}
        except Exception:
            continue
    # pick threshold maximizing F1 on validation
    p, r, t = precision_recall_curve(y_val, best_if['val_scores'])
    f1 = 2*p*r/(p+r+1e-12)
    best_idx = np.nanargmax(f1)
    best_if_threshold = t[best_idx] if best_idx < len(t) else t[-1]
    # evaluate on test
    test_scores_if = -best_if['model'].decision_function(Xt)
    test_ap_if = average_precision_score(y_test, test_scores_if >= best_if_threshold)
    test_roc_if = roc_auc_score(y_test, test_scores_if)
    results[atk][f'{prefix}_if_tuned_model'] = best_if['model']
    results[atk][f'{prefix}_if_tuned_threshold'] = best_if_threshold
    results[atk][f'{prefix}_if_tuned_ap_test'] = test_ap_if
    results[atk][f'{prefix}_if_tuned_roc_test'] = test_roc_if

    # 2) OCSVM quick grid
    best_ocsvm = {'ap': -1}
    # train on subset if large
    tr_idx = np.arange(len(Xtr))
    if len(Xtr) > 5000:
        rng = np.random.RandomState(42)
        tr_idx = rng.choice(len(Xtr), 5000, replace=False)
    Xtr_ocsvm = Xtr[tr_idx]
    for nu in [0.005, 0.01, 0.02]:
        try:
            m = OneClassSVM(nu=nu, kernel='rbf', gamma='scale')
            m.fit(Xtr_ocsvm)
            val_scores = -m.decision_function(Xv)
            ap = average_precision_score(y_val, val_scores)
            if ap > best_ocsvm['ap']:
                best_ocsvm = {'ap':ap,'model':m,'nu':nu,'val_scores':val_scores}
        except Exception:
            continue
    if best_ocsvm['ap'] >= 0:
        p, r, t = precision_recall_curve(y_val, best_ocsvm['val_scores'])
        f1 = 2*p*r/(p+r+1e-12)
        best_idx = np.nanargmax(f1)
        best_ocsvm_threshold = t[best_idx] if best_idx < len(t) else t[-1]
        test_scores_ocsvm = -best_ocsvm['model'].decision_function(Xt)
        test_ap_ocsvm = average_precision_score(y_test, test_scores_ocsvm >= best_ocsvm_threshold)
        test_roc_ocsvm = roc_auc_score(y_test, test_scores_ocsvm)
        results[atk][f'{prefix}_ocsvm_tuned_model'] = best_ocsvm['model']
        results[atk][f'{prefix}_ocsvm_tuned_threshold'] = best_ocsvm_threshold
        results[atk][f'{prefix}_ocsvm_tuned_ap_test'] = test_ap_ocsvm
        results[atk][f'{prefix}_ocsvm_tuned_roc_test'] = test_roc_ocsvm

    # 3) LOF quick grid (novelty=True so we can call decision_function)
    best_lof = {'ap': -1}
    for n_neighbors in [10, 15, 30]:
        try:
            m = LocalOutlierFactor(n_neighbors=n_neighbors, novelty=True, contamination='auto')
            # fit on subset if needed
            m.fit(Xtr if len(Xtr) <= 20000 else Xtr[np.random.RandomState(42).choice(len(Xtr), 10000, replace=False)])
            val_scores = -m.decision_function(Xv)
            ap = average_precision_score(y_val, val_scores)
            if ap > best_lof['ap']:
                best_lof = {'ap':ap,'model':m,'n_neighbors':n_neighbors,'val_scores':val_scores}
        except Exception:
            continue
    if best_lof['ap'] >= 0:
        p, r, t = precision_recall_curve(y_val, best_lof['val_scores'])
        f1 = 2*p*r/(p+r+1e-12)
        best_idx = np.nanargmax(f1)
        best_lof_threshold = t[best_idx] if best_idx < len(t) else t[-1]
        test_scores_lof = -best_lof['model'].decision_function(Xt)
        test_ap_lof = average_precision_score(y_test, test_scores_lof >= best_lof_threshold)
        test_roc_lof = roc_auc_score(y_test, test_scores_lof)
        results[atk][f'{prefix}_lof_tuned_model'] = best_lof['model']
        results[atk][f'{prefix}_lof_tuned_threshold'] = best_lof_threshold
        results[atk][f'{prefix}_lof_tuned_ap_test'] = test_ap_lof
        results[atk][f'{prefix}_lof_tuned_roc_test'] = test_roc_lof

    best_summary.append((prefix,
                         results[atk].get(f'{prefix}_if_tuned_ap_test'),
                         results[atk].get(f'{prefix}_ocsvm_tuned_ap_test'),
                         results[atk].get(f'{prefix}_lof_tuned_ap_test')))

# Print small table of test AP after tuning
import pandas as pd
df = pd.DataFrame(best_summary, columns=['prefix','IF_test_AP','OCSVM_test_AP','LOF_test_AP'])
print('Port_scan tuned test AP (by prefix):')
print(df.to_string(index=False))

# Task
Implement hyperparameter tuning for Isolation Forest, OCSVM, and LOF models by defining parameter grids for each model. For each attack type and feature selection method, iterate through these grids, train the models on the training data, and evaluate their performance using Average Precision on the validation sets. Store the best models and their optimized hyperparameters in the `results` dictionary. After tuning, re-evaluate the performance of all models (Isolation Forest, Autoencoder, OCSVM, and LOF) on their respective test sets, using the newly optimized parameters for Isolation Forest, OCSVM, and LOF. Calculate and store the ROC AUC and Average Precision scores for all models on the test sets. Finally, update the grouped bar charts to visualize the performance of all models, including OCSVM and LOF, and update the visualization for the overall best-performing model based on the tuned parameters. Summarize the performance of all anomaly detection models, highlighting the impact of hyperparameter tuning and proposing future steps.

## Implement Hyperparameter Tuning

### Subtask:
Implement hyperparameter tuning for Isolation Forest, One-Class SVM (OCSVM), and Local Outlier Factor (LOF) models using Average Precision as the scoring metric on validation sets. Store the best models and their optimized hyperparameters in the `results` dictionary.


**Reasoning**:
I will define parameter grids for Isolation Forest, One-Class SVM, and Local Outlier Factor (LOF) models. Then, I will iterate through each attack type and feature selection method, retrieving the data splits. For each combination, I will check for sufficient anomalies in the validation set. Subsequently, I will perform a manual grid search for each model, fit them on the training data, calculate anomaly scores on the validation set, and evaluate performance using Average Precision. Finally, I will store the best-performing models and their hyperparameters in the `results` dictionary and print the tuning results. I will explicitly handle the `contamination` parameter constraint for LOF with `novelty=True`.



In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import average_precision_score
import numpy as np
import joblib
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# Optimized parameter grids (reduced for speed)
param_grids = {
    'Isolation Forest': {
        'n_estimators': [100],  # Reduced: Single value for speed
        'max_samples': [256, 512],  # Added: Controls tree size
        'contamination': [0.005, 0.01]  # Reduced: Focus on realistic values
    },
    'One-Class SVM': {
        'nu': [0.005, 0.01],  # Reduced: Most common values
        'kernel': ['rbf'],  # Single best kernel
        'gamma': ['scale']  # Single best gamma
    },
    'Local Outlier Factor': {
        'n_neighbors': [10, 20],  # Reduced
        'algorithm': ['auto']  # Added for speed
    }
}

print("Starting OPTIMIZED hyperparameter tuning...")

# Configuration
MAX_SAMPLES = 2000  # Reduced from 5000
MAX_VAL_SAMPLES = 1000  # Score on subset
USE_PARALLEL = True  # Parallelize across attack types
N_JOBS = 2  # Conservative for Colab CPU

# Store results
tuned_models_results = {}

def tune_model_for_config(args):
    """Parallel worker function"""
    attack_type, prefix, X_train, X_val, y_val = args

    results = {
        'if_tuned_model': None,
        'if_tuned_params': None,
        'ocsvm_tuned_model': None,
        'ocsvm_tuned_params': None,
        'lof_tuned_model': None,
        'lof_tuned_params': None
    }

    # Skip if insufficient anomalies
    num_anomalies = np.sum(y_val == 1)
    if num_anomalies < 1:
        return (attack_type, prefix), results

    print(f"  Tuning {attack_type} - {prefix}...")

    # --- 1. OPTIMIZATION: Subsample validation for scoring ---
    if len(X_val) > MAX_VAL_SAMPLES:
        idx = np.random.RandomState(42).choice(len(X_val), MAX_VAL_SAMPLES, replace=False)
        X_val_scoring = X_val[idx]
        y_val_scoring = y_val[idx]
    else:
        X_val_scoring = X_val
        y_val_scoring = y_val

    # --- 2. OPTIMIZATION: Standardize data once for all models ---
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val_scoring)

    # --- Isolation Forest Tuning ---
    print(f"    IF for {attack_type} - {prefix}...", end="")
    best_if_ap = -1

    # OPTIMIZATION: Pre-compute sample size
    n_samples = min(256, len(X_train_scaled))

    for n_est in param_grids['Isolation Forest']['n_estimators']:
        for max_samp in param_grids['Isolation Forest']['max_samples']:
            for contam in param_grids['Isolation Forest']['contamination']:
                try:
                    # OPTIMIZATION: Use faster parameters
                    if_model = IsolationForest(
                        n_estimators=n_est,
                        max_samples=max_samp,
                        contamination=contam,
                        max_features=0.5,  # Reduced for speed
                        bootstrap=False,  # Faster training
                        random_state=42,
                        n_jobs=1  # Single job for stability
                    )
                    # OPTIMIZATION: Train on subset for speed
                    if len(X_train_scaled) > MAX_SAMPLES:
                        idx = np.random.RandomState(42).choice(
                            len(X_train_scaled),
                            MAX_SAMPLES,
                            replace=False
                        )
                        if_model.fit(X_train_scaled[idx])
                    else:
                        if_model.fit(X_train_scaled)

                    # OPTIMIZATION: Fast scoring
                    val_scores_if = if_model.score_samples(X_val_scaled)  # Faster than decision_function
                    ap_score_if = average_precision_score(y_val_scoring, -val_scores_if)

                    if ap_score_if > best_if_ap:
                        best_if_ap = ap_score_if
                        results['if_tuned_model'] = if_model
                        results['if_tuned_params'] = {
                            'n_estimators': n_est,
                            'max_samples': max_samp,
                            'contamination': contam
                        }
                except Exception:
                    continue
    print(f" AP: {best_if_ap:.4f}")

    # --- One-Class SVM Tuning ---
    print(f"    OCSVM for {attack_type} - {prefix}...", end="")
    best_ocsvm_ap = -1

    # OPTIMIZATION: Train OCSVM on subset
    if len(X_train_scaled) > MAX_SAMPLES:
        idx = np.random.RandomState(42).choice(
            len(X_train_scaled),
            min(MAX_SAMPLES, len(X_train_scaled)),
            replace=False
        )
        X_train_ocsvm = X_train_scaled[idx]
    else:
        X_train_ocsvm = X_train_scaled

    for nu_val in param_grids['One-Class SVM']['nu']:
        for kernel_val in param_grids['One-Class SVM']['kernel']:
            for gamma_val in param_grids['One-Class SVM']['gamma']:
                try:
                    # OPTIMIZATION: Smaller cache size
                    ocsvm_model = OneClassSVM(
                        nu=nu_val,
                        kernel=kernel_val,
                        gamma=gamma_val,
                        cache_size=200,  # Reduced for memory
                        max_iter=1000,  # Limit iterations
                        verbose=False
                    )
                    ocsvm_model.fit(X_train_ocsvm)

                    # OPTIMIZATION: Score in batches
                    val_scores_ocsvm = ocsvm_model.decision_function(X_val_scaled)
                    ap_score_ocsvm = average_precision_score(y_val_scoring, -val_scores_ocsvm)

                    if ap_score_ocsvm > best_ocsvm_ap:
                        best_ocsvm_ap = ap_score_ocsvm
                        results['ocsvm_tuned_model'] = ocsvm_model
                        results['ocsvm_tuned_params'] = {
                            'nu': nu_val,
                            'kernel': kernel_val,
                            'gamma': gamma_val
                        }
                except Exception:
                    continue
    print(f" AP: {best_ocsvm_ap:.4f}")

    # --- Local Outlier Factor Tuning ---
    print(f"    LOF for {attack_type} - {prefix}...", end="")
    best_lof_ap = -1

    # OPTIMIZATION: Further reduce LOF training samples
    lof_max_samples = min(1000, len(X_train_scaled))
    if len(X_train_scaled) > lof_max_samples:
        idx = np.random.RandomState(42).choice(
            len(X_train_scaled),
            lof_max_samples,
            replace=False
        )
        X_train_lof = X_train_scaled[idx]
    else:
        X_train_lof = X_train_scaled

    for n_neigh in param_grids['Local Outlier Factor']['n_neighbors']:
        for algo in param_grids['Local Outlier Factor']['algorithm']:
            if n_neigh >= len(X_train_lof):
                continue

            try:
                # OPTIMIZATION: Optimized LOF parameters
                lof_model = LocalOutlierFactor(
                    n_neighbors=n_neigh,
                    algorithm=algo,
                    leaf_size=20,  # Smaller for faster queries
                    novelty=True,
                    contamination='auto',
                    n_jobs=1  # Single job
                )
                lof_model.fit(X_train_lof)

                # OPTIMIZATION: Score validation subset
                val_scores_lof = lof_model.decision_function(X_val_scaled)
                ap_score_lof = average_precision_score(y_val_scoring, -val_scores_lof)

                if ap_score_lof > best_lof_ap:
                    best_lof_ap = ap_score_lof
                    results['lof_tuned_model'] = lof_model
                    results['lof_tuned_params'] = {
                        'n_neighbors': n_neigh,
                        'algorithm': algo,
                        'contamination': 'auto'
                    }
            except Exception:
                continue
    print(f" AP: {best_lof_ap:.4f}")

    # Store scaler for later use
    results['scaler'] = scaler

    return (attack_type, prefix), results

# Prepare all configurations
configs = []
for attack_type, methods in results.items():
    for prefix in ['heuristic', 'variance', 'pca']:
        X_train = methods.get(f'{prefix}_X_train')
        X_val = methods.get(f'{prefix}_X_val')
        y_val = methods.get(f'{prefix}_y_val')

        if (X_train is not None and len(X_train) > 10 and
            X_val is not None and len(X_val) > 10 and
            y_val is not None and np.sum(y_val == 1) > 0):
            configs.append((attack_type, prefix, X_train, X_val, y_val))

# Run tuning (parallel or sequential)
print(f"\nFound {len(configs)} configurations to tune")
tuned_results = {}

if USE_PARALLEL and len(configs) > 1:
    print("Running parallel tuning...")
    with ThreadPoolExecutor(max_workers=N_JOBS) as executor:
        future_to_config = {
            executor.submit(tune_model_for_config, config): config
            for config in configs
        }

        for future in as_completed(future_to_config):
            config_key, config_results = future.result()
            tuned_results[config_key] = config_results
            attack_type, prefix = config_key
            print(f"  Completed {attack_type} - {prefix}")
else:
    print("Running sequential tuning...")
    for config in configs:
        config_key, config_results = tune_model_for_config(config)
        tuned_results[config_key] = config_results

# Organize results back into main structure
for (attack_type, prefix), model_results in tuned_results.items():
    if attack_type not in tuned_models_results:
        tuned_models_results[attack_type] = {}
    tuned_models_results[attack_type][prefix] = model_results

# Update main results dictionary
update_count = 0
for attack_type in tuned_models_results:
    for prefix in tuned_models_results[attack_type]:
        for key, value in tuned_models_results[attack_type][prefix].items():
            if key != 'scaler':  # Store scaler separately
                results[attack_type][f'{prefix}_{key}'] = value
                update_count += 1
            else:
                results[attack_type][f'{prefix}_scaler'] = value

print(f"\nHyperparameter tuning complete!")
print(f"Updated {update_count} model entries in results dictionary")
print(f"Total configurations tuned: {len(tuned_results)}")

# Task
Evaluate all models with their tuned parameters (where applicable) on their respective test sets, calculating and storing ROC AUC and Average Precision scores for Isolation Forest, Autoencoder, OCSVM, and LOF models.

## Evaluate All Models with Tuned Parameters

### Subtask:
Re-evaluate the performance of all models (Isolation Forest, Autoencoder, OCSVM, and LOF) on their respective test sets, using the parameters optimized during the hyperparameter tuning step for Isolation Forest, OCSVM, and LOF. Calculate and store ROC AUC and Average Precision scores for all models.


**Reasoning**:
I will re-evaluate all models (Isolation Forest, Autoencoder, OCSVM, and LOF) on their respective test sets. For Isolation Forest, OCSVM, and LOF, I will prioritize using the tuned models and their corresponding scalers. For Autoencoders, I will use their previously calculated test reconstruction errors. I will then calculate and store the ROC AUC and Average Precision scores for all models in the `results` dictionary.



In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

print("Re-evaluating all models with (tuned) parameters on test data...")

# Helper function to get model and scale X_test
def get_model_and_scaled_data(attack_type, prefix, model_type, X_test, results):
    model = None
    scaler = results[attack_type].get(f'{prefix}_scaler')

    # Prioritize tuned model
    tuned_model_key = f'{prefix}_{model_type}_tuned_model'
    if tuned_model_key in results[attack_type]:
        model = results[attack_type][tuned_model_key]
    else:
        # Fallback to untuned model
        untuned_model_key = f'{prefix}_{model_type}_model'
        if untuned_model_key in results[attack_type]:
            model = results[attack_type][untuned_model_key]

    scaled_X_test = X_test
    if scaler is not None:
        scaled_X_test = scaler.transform(X_test)

    return model, scaled_X_test

for attack_type, methods in results.items():
    for prefix in ['heuristic', 'variance', 'pca']:
        print(f"  Processing {attack_type.replace('_', ' ').title()} - {prefix.title()}...")

        X_test = methods.get(f'{prefix}_X_test')
        y_test = methods.get(f'{prefix}_y_test')

        if X_test is None or y_test is None or len(X_test) == 0:
            print(f"    Skipping: Missing test data for {attack_type} - {prefix}")
            continue

        # Ensure y_test has both classes for metric calculation
        if len(np.unique(y_test)) < 2:
            print(f"    Skipping metrics: Single class in y_test for {attack_type} - {prefix}")
            continue

        # --- Isolation Forest ---
        if_model, scaled_X_test_if = get_model_and_scaled_data(attack_type, prefix, 'if', X_test, results)
        if if_model:
            if_test_scores = if_model.decision_function(scaled_X_test_if)
            results[attack_type][f'{prefix}_if_roc_auc_tuned'] = roc_auc_score(y_test, if_test_scores)
            results[attack_type][f'{prefix}_if_avg_precision_tuned'] = average_precision_score(y_test, if_test_scores)
            print(f"      IF ROC AUC (Tuned): {results[attack_type][f'{prefix}_if_roc_auc_tuned']:.4f}")
        else:
            print(f"      No IF model found for {attack_type} - {prefix}")

        # --- Autoencoder --- (scores are already computed, just re-evaluate metrics consistently)
        ae_test_errors_key = f'{prefix}_test_reconstruction_errors'
        if ae_test_errors_key in methods:
            ae_test_errors = methods[ae_test_errors_key]
            results[attack_type][f'{prefix}_ae_roc_auc_tuned'] = roc_auc_score(y_test, ae_test_errors)
            results[attack_type][f'{prefix}_ae_avg_precision_tuned'] = average_precision_score(y_test, ae_test_errors)
            print(f"      AE ROC AUC (Tuned): {results[attack_type][f'{prefix}_ae_roc_auc_tuned']:.4f}")
        else:
            print(f"      No AE errors found for {attack_type} - {prefix}")

        # --- One-Class SVM (OCSVM) ---
        ocsvm_model, scaled_X_test_ocsvm = get_model_and_scaled_data(attack_type, prefix, 'ocsvm', X_test, results)
        if ocsvm_model:
            ocsvm_test_scores = ocsvm_model.decision_function(scaled_X_test_ocsvm)
            results[attack_type][f'{prefix}_ocsvm_roc_auc_tuned'] = roc_auc_score(y_test, ocsvm_test_scores)
            results[attack_type][f'{prefix}_ocsvm_avg_precision_tuned'] = average_precision_score(y_test, ocsvm_test_scores)
            print(f"      OCSVM ROC AUC (Tuned): {results[attack_type][f'{prefix}_ocsvm_roc_auc_tuned']:.4f}")
        else:
            print(f"      No OCSVM model found for {attack_type} - {prefix}")

        # --- Local Outlier Factor (LOF) ---
        # Note: For LOF with novelty=True, fit on X_train, then decision_function on new data
        # If novelty=False, fit_predict on the whole dataset. Here we assume novelty=True from tuning.
        lof_model, scaled_X_test_lof = get_model_and_scaled_data(attack_type, prefix, 'lof', X_test, results)
        if lof_model:
            lof_test_scores = lof_model.decision_function(scaled_X_test_lof)
            results[attack_type][f'{prefix}_lof_roc_auc_tuned'] = roc_auc_score(y_test, lof_test_scores)
            results[attack_type][f'{prefix}_lof_avg_precision_tuned'] = average_precision_score(y_test, lof_test_scores)
            print(f"      LOF ROC AUC (Tuned): {results[attack_type][f'{prefix}_lof_roc_auc_tuned']:.4f}")
        else:
            print(f"      No LOF model found for {attack_type} - {prefix}")

print("\nAll models re-evaluated. Tuned metrics stored in results dictionary.")


**Reasoning**:
I will update the grouped bar charts to include the performance metrics (ROC AUC) for the newly evaluated OCSVM and LOF models, alongside Isolation Forest and Autoencoder. This will provide a comprehensive visualization of all models' performance after tuning.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize an empty list to store performance data for ROC AUC
performance_data_roc_auc = []

# Define selection method prefixes for iteration consistency
selection_method_prefixes = ['heuristic', 'variance', 'pca']

# Iterate through each attack_type and its methods in the results dictionary
for attack_type, methods in results.items():
    # For each attack_type, iterate through the selection_method_prefixes
    for prefix in selection_method_prefixes:
        # Check for Isolation Forest ROC AUC scores (tuned)
        if_roc_auc_key = f'{prefix}_if_roc_auc_tuned'
        if if_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Isolation Forest',
                'Metric': 'ROC AUC',
                'Score': methods[if_roc_auc_key]
            })

        # Check for Autoencoder ROC AUC scores (tuned - using previous values)
        ae_roc_auc_key = f'{prefix}_ae_roc_auc_tuned'
        if ae_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Autoencoder',
                'Metric': 'ROC AUC',
                'Score': methods[ae_roc_auc_key]
            })

        # Check for OCSVM ROC AUC scores (tuned)
        ocsvm_roc_auc_key = f'{prefix}_ocsvm_roc_auc_tuned'
        if ocsvm_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'OCSVM',
                'Metric': 'ROC AUC',
                'Score': methods[ocsvm_roc_auc_key]
            })

        # Check for LOF ROC AUC scores (tuned)
        lof_roc_auc_key = f'{prefix}_lof_roc_auc_tuned'
        if lof_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'LOF',
                'Metric': 'ROC AUC',
                'Score': methods[lof_roc_auc_key]
            })

# Convert the list of performance data into a Pandas DataFrame
roc_auc_df_tuned = pd.DataFrame(performance_data_roc_auc)

# Create a grouped bar chart using seaborn.catplot for ROC AUC
g_roc_auc = sns.catplot(data=roc_auc_df_tuned, x='Attack Type', y='Score', hue='Model', col='Feature Selection',
                kind='bar', palette='viridis', dodge=True, col_wrap=3, height=5, aspect=1.2)

# Add appropriate labels and a title
g_roc_auc.set_axis_labels('Attack Type', 'ROC AUC Score')
g_roc_auc.set_titles('Feature Selection: {col_name}')
g_roc_auc.set_xticklabels(rotation=45, ha='right')
plt.suptitle('Tuned Model ROC AUC Score Comparison Across Attack Types, Feature Selection, and Models', y=1.02, fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap

# Display the plot
plt.show()


**Reasoning**:
I will now update the grouped bar charts to include the Average Precision (AP) for the newly evaluated OCSVM and LOF models, alongside Isolation Forest and Autoencoder. This will provide a comprehensive visualization of all models' performance after tuning, addressing the second part of the visualization subtask.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize an empty list to store performance data for Average Precision
performance_data_avg_precision = []

# Define selection method prefixes for iteration consistency
selection_method_prefixes = ['heuristic', 'variance', 'pca']

# Iterate through each attack_type and its methods in the results dictionary
for attack_type, methods in results.items():
    # For each attack_type, iterate through the selection_method_prefixes
    for prefix in selection_method_prefixes:
        # Check for Isolation Forest Average Precision scores (tuned)
        if_avg_precision_key = f'{prefix}_if_avg_precision_tuned'
        if if_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Isolation Forest',
                'Metric': 'Average Precision',
                'Score': methods[if_avg_precision_key]
            })

        # Check for Autoencoder Average Precision scores (tuned - using previous values)
        ae_avg_precision_key = f'{prefix}_ae_avg_precision_tuned'
        if ae_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Autoencoder',
                'Metric': 'Average Precision',
                'Score': methods[ae_avg_precision_key]
            })

        # Check for OCSVM Average Precision scores (tuned)
        ocsvm_avg_precision_key = f'{prefix}_ocsvm_avg_precision_tuned'
        if ocsvm_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'OCSVM',
                'Metric': 'Average Precision',
                'Score': methods[ocsvm_avg_precision_key]
            })

        # Check for LOF Average Precision scores (tuned)
        lof_avg_precision_key = f'{prefix}_lof_avg_precision_tuned'
        if lof_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'LOF',
                'Metric': 'Average Precision',
                'Score': methods[lof_avg_precision_key]
            })

# Convert the list of performance data into a Pandas DataFrame
avg_precision_df_tuned = pd.DataFrame(performance_data_avg_precision)

# Create a grouped bar chart using seaborn.catplot for Average Precision
g_avg_precision = sns.catplot(data=avg_precision_df_tuned, x='Attack Type', y='Score', hue='Model', col='Feature Selection',
                kind='bar', palette='viridis', dodge=True, col_wrap=3, height=5, aspect=1.2)

# Add appropriate labels and a title
g_avg_precision.set_axis_labels('Attack Type', 'Average Precision Score')
g_avg_precision.set_titles('Feature Selection: {col_name}')
g_avg_precision.set_xticklabels(rotation=45, ha='right')
plt.suptitle('Tuned Model Average Precision Score Comparison Across Attack Types, Feature Selection, and Models', y=1.02, fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap

# Display the plot
plt.show()


**Reasoning**:
I will update the visualization for the best performing model. First, I need to iterate through the `results` dictionary to find the model (Isolation Forest, Autoencoder, OCSVM, or LOF) that achieved the highest ROC AUC score after tuning across all attack types and feature selection methods. Then, I will retrieve the relevant test scores (anomaly scores or reconstruction errors) and the corresponding ground truth labels for this best model and create a scatter plot to visualize its performance.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# 1. Initialize variables to store the best ROC AUC score and its details
best_roc_auc_tuned = -1
best_attack_type = None
best_selection_method = None
best_model_type = None

# 2. Iterate through the results dictionary to find the combination with the highest ROC AUC score (tuned)
for attack_type, methods in results.items():
    for prefix in ['heuristic', 'variance', 'pca']:
        # Check Isolation Forest ROC AUC (tuned)
        if_roc_auc_key = f'{prefix}_if_roc_auc_tuned'
        if if_roc_auc_key in methods:
            score = methods[if_roc_auc_key]
            if score > best_roc_auc_tuned:
                best_roc_auc_tuned = score
                best_attack_type = attack_type
                best_selection_method = prefix
                best_model_type = 'Isolation Forest'

        # Check Autoencoder ROC AUC (tuned)
        ae_roc_auc_key = f'{prefix}_ae_roc_auc_tuned'
        if ae_roc_auc_key in methods:
            score = methods[ae_roc_auc_key]
            if score > best_roc_auc_tuned:
                best_roc_auc_tuned = score
                best_attack_type = attack_type
                best_selection_method = prefix
                best_model_type = 'Autoencoder'

        # Check OCSVM ROC AUC (tuned)
        ocsvm_roc_auc_key = f'{prefix}_ocsvm_roc_auc_tuned'
        if ocsvm_roc_auc_key in methods:
            score = methods[ocsvm_roc_auc_key]
            if score > best_roc_auc_tuned:
                best_roc_auc_tuned = score
                best_attack_type = attack_type
                best_selection_method = prefix
                best_model_type = 'OCSVM'

        # Check LOF ROC AUC (tuned)
        lof_roc_auc_key = f'{prefix}_lof_roc_auc_tuned'
        if lof_roc_auc_key in methods:
            score = methods[lof_roc_auc_key]
            if score > best_roc_auc_tuned:
                best_roc_auc_tuned = score
                best_attack_type = attack_type
                best_selection_method = prefix
                best_model_type = 'LOF'

print(f"Best Tuned Model Found: {best_model_type} for {best_attack_type.replace('_', ' ').title()} with {best_selection_method.title()} features (ROC AUC: {best_roc_auc_tuned:.4f})")

# 3. Retrieve the test anomaly scores/reconstruction errors and y_test labels for the best combination
if best_model_type == 'Isolation Forest':
    # Use the tuned model to get decision function if it exists, otherwise fall back to untuned
    if f'{best_selection_method}_if_tuned_model' in results[best_attack_type]:
        model = results[best_attack_type][f'{best_selection_method}_if_tuned_model']
    else:
        model = results[best_attack_type][f'{best_selection_method}_isolation_forest_model']

    # Scale X_test if scaler exists
    X_test_original = results[best_attack_type][f'{best_selection_method}_X_test']
    scaler = results[best_attack_type].get(f'{best_selection_method}_scaler')
    if scaler is not None:
        X_test_scaled = scaler.transform(X_test_original)
    else:
        X_test_scaled = X_test_original

    anomaly_scores = model.decision_function(X_test_scaled)
    metric_label = 'Anomaly Score'
elif best_model_type == 'Autoencoder':
    anomaly_scores = results[best_attack_type][f'{best_selection_method}_test_reconstruction_errors']
    metric_label = 'Reconstruction Error'
elif best_model_type == 'OCSVM':
    if f'{best_selection_method}_ocsvm_tuned_model' in results[best_attack_type]:
        model = results[best_attack_type][f'{best_selection_method}_ocsvm_tuned_model']
    else:
        model = results[best_attack_type][f'{best_selection_method}_ocsvm_model']

    X_test_original = results[best_attack_type][f'{best_selection_method}_X_test']
    scaler = results[best_attack_type].get(f'{best_selection_method}_scaler')
    if scaler is not None:
        X_test_scaled = scaler.transform(X_test_original)
    else:
        X_test_scaled = X_test_original
    anomaly_scores = model.decision_function(X_test_scaled)
    metric_label = 'Anomaly Score'
elif best_model_type == 'LOF':
    if f'{best_selection_method}_lof_tuned_model' in results[best_attack_type]:
        model = results[best_attack_type][f'{best_selection_method}_lof_tuned_model']
    else:
        model = results[best_attack_type][f'{best_selection_method}_lof_model']

    X_test_original = results[best_attack_type][f'{best_selection_method}_X_test']
    scaler = results[best_attack_type].get(f'{best_selection_method}_scaler')
    if scaler is not None:
        X_test_scaled = scaler.transform(X_test_original)
    else:
        X_test_scaled = X_test_original
    anomaly_scores = model.decision_function(X_test_scaled)
    metric_label = 'Anomaly Score'

y_test_key = f'{best_selection_method}_y_test'
y_test = results[best_attack_type][y_test_key]

# Create a DataFrame for plotting
plot_df = pd.DataFrame({
    metric_label: anomaly_scores,
    'Ground Truth': y_test
})

# Map ground truth labels to more descriptive names for the legend
plot_df['Ground Truth Label'] = plot_df['Ground Truth'].map({0: 'Normal', 1: 'Anomaly'})

# 4. Create a scatter plot
plt.figure(figsize=(12, 7))
sns.scatterplot(
    data=plot_df,
    x=np.random.rand(len(plot_df)) * (plot_df.index.max() - plot_df.index.min()) + plot_df.index.min(), # Jittered x-axis
    y=metric_label,
    hue='Ground Truth Label',
    palette={'Normal': 'skyblue', 'Anomaly': 'red'},
    alpha=0.6,
    s=10 # Smaller point size for large datasets
)

# 5. Add appropriate labels and a title
plt.xlabel('Data Point Index (Jittered)')
plt.ylabel(metric_label)
plt.title(f'{best_model_type} Anomaly Detection for {best_attack_type.replace('_', ' ').title()} (Features: {best_selection_method.title()})\nROC AUC: {best_roc_auc_tuned:.4f}')
plt.legend(title='Ground Truth')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

## Evaluate All Models with Tuned Parameters

### Subtask:
Re-evaluate the performance of all models (Isolation Forest, Autoencoder, OCSVM, and LOF) on their respective test sets, using the parameters optimized during the hyperparameter tuning step for Isolation Forest, OCSVM, and LOF. Calculate and store ROC AUC and Average Precision scores for all models.


**Reasoning**:
I will re-evaluate all models (Isolation Forest, Autoencoder, OCSVM, and LOF) on their respective test sets. For Isolation Forest, OCSVM, and LOF, I will prioritize using the tuned models and their corresponding scalers. For Autoencoders, I will use their previously calculated test reconstruction errors. I will then calculate and store the ROC AUC and Average Precision scores for all models in the `results` dictionary.



In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

print("Re-evaluating all models with (tuned) parameters on test data...")

# Helper function to get model and scale X_test
def get_model_and_scaled_data(attack_type, prefix, model_type, X_test, results):
    model = None
    scaler = results[attack_type].get(f'{prefix}_scaler')

    # Prioritize tuned model
    tuned_model_key = f'{prefix}_{model_type}_tuned_model'
    if tuned_model_key in results[attack_type]:
        model = results[attack_type][tuned_model_key]
    else:
        # Fallback to untuned model
        untuned_model_key = f'{prefix}_{model_type}_model'
        if untuned_model_key in results[attack_type]:
            model = results[attack_type][untuned_model_key]

    scaled_X_test = X_test
    if scaler is not None:
        scaled_X_test = scaler.transform(X_test)

    return model, scaled_X_test

for attack_type, methods in results.items():
    for prefix in ['heuristic', 'variance', 'pca']:
        print(f"  Processing {attack_type.replace('_', ' ').title()} - {prefix.title()}...")

        X_test = methods.get(f'{prefix}_X_test')
        y_test = methods.get(f'{prefix}_y_test')

        if X_test is None or y_test is None or len(X_test) == 0:
            print(f"    Skipping: Missing test data for {attack_type} - {prefix}")
            continue

        # Ensure y_test has both classes for metric calculation
        if len(np.unique(y_test)) < 2:
            print(f"    Skipping metrics: Single class in y_test for {attack_type} - {prefix}")
            continue

        # --- Isolation Forest ---
        if_model, scaled_X_test_if = get_model_and_scaled_data(attack_type, prefix, 'if', X_test, results)
        if if_model:
            if_test_scores = if_model.decision_function(scaled_X_test_if)
            results[attack_type][f'{prefix}_if_roc_auc_tuned'] = roc_auc_score(y_test, if_test_scores)
            results[attack_type][f'{prefix}_if_avg_precision_tuned'] = average_precision_score(y_test, if_test_scores)
            print(f"      IF ROC AUC (Tuned): {results[attack_type][f'{prefix}_if_roc_auc_tuned']:.4f}")
        else:
            print(f"      No IF model found for {attack_type} - {prefix}")

        # --- Autoencoder --- (scores are already computed, just re-evaluate metrics consistently)
        ae_test_errors_key = f'{prefix}_test_reconstruction_errors'
        if ae_test_errors_key in methods:
            ae_test_errors = methods[ae_test_errors_key]
            results[attack_type][f'{prefix}_ae_roc_auc_tuned'] = roc_auc_score(y_test, ae_test_errors)
            results[attack_type][f'{prefix}_ae_avg_precision_tuned'] = average_precision_score(y_test, ae_test_errors)
            print(f"      AE ROC AUC (Tuned): {results[attack_type][f'{prefix}_ae_roc_auc_tuned']:.4f}")
        else:
            print(f"      No AE errors found for {attack_type} - {prefix}")

        # --- One-Class SVM (OCSVM) ---
        ocsvm_model, scaled_X_test_ocsvm = get_model_and_scaled_data(attack_type, prefix, 'ocsvm', X_test, results)
        if ocsvm_model:
            ocsvm_test_scores = ocsvm_model.decision_function(scaled_X_test_ocsvm)
            results[attack_type][f'{prefix}_ocsvm_roc_auc_tuned'] = roc_auc_score(y_test, ocsvm_test_scores)
            results[attack_type][f'{prefix}_ocsvm_avg_precision_tuned'] = average_precision_score(y_test, ocsvm_test_scores)
            print(f"      OCSVM ROC AUC (Tuned): {results[attack_type][f'{prefix}_ocsvm_roc_auc_tuned']:.4f}")
        else:
            print(f"      No OCSVM model found for {attack_type} - {prefix}")

        # --- Local Outlier Factor (LOF) ---
        # Note: For LOF with novelty=True, fit on X_train, then decision_function on new data
        # If novelty=False, fit_predict on the whole dataset. Here we assume novelty=True from tuning.
        lof_model, scaled_X_test_lof = get_model_and_scaled_data(attack_type, prefix, 'lof', X_test, results)
        if lof_model:
            lof_test_scores = lof_model.decision_function(scaled_X_test_lof)
            results[attack_type][f'{prefix}_lof_roc_auc_tuned'] = roc_auc_score(y_test, lof_test_scores)
            results[attack_type][f'{prefix}_lof_avg_precision_tuned'] = average_precision_score(y_test, lof_test_scores)
            print(f"      LOF ROC AUC (Tuned): {results[attack_type][f'{prefix}_lof_roc_auc_tuned']:.4f}")
        else:
            print(f"      No LOF model found for {attack_type} - {prefix}")

print("\nAll models re-evaluated. Tuned metrics stored in results dictionary.")


**Reasoning**:
I will now update the grouped bar charts to include the performance metrics (ROC AUC) for the newly evaluated OCSVM and LOF models, alongside Isolation Forest and Autoencoder. This will provide a comprehensive visualization of all models' performance after tuning.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize an empty list to store performance data for ROC AUC
performance_data_roc_auc = []

# Define selection method prefixes for iteration consistency
selection_method_prefixes = ['heuristic', 'variance', 'pca']

# Iterate through each attack_type and its methods in the results dictionary
for attack_type, methods in results.items():
    # For each attack_type, iterate through the selection_method_prefixes
    for prefix in selection_method_prefixes:
        # Check for Isolation Forest ROC AUC scores (tuned)
        if_roc_auc_key = f'{prefix}_if_roc_auc_tuned'
        if if_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Isolation Forest',
                'Metric': 'ROC AUC',
                'Score': methods[if_roc_auc_key]
            })

        # Check for Autoencoder ROC AUC scores (tuned - using previous values)
        ae_roc_auc_key = f'{prefix}_ae_roc_auc_tuned'
        if ae_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Autoencoder',
                'Metric': 'ROC AUC',
                'Score': methods[ae_roc_auc_key]
            })

        # Check for OCSVM ROC AUC scores (tuned)
        ocsvm_roc_auc_key = f'{prefix}_ocsvm_roc_auc_tuned'
        if ocsvm_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'OCSVM',
                'Metric': 'ROC AUC',
                'Score': methods[ocsvm_roc_auc_key]
            })

        # Check for LOF ROC AUC scores (tuned)
        lof_roc_auc_key = f'{prefix}_lof_roc_auc_tuned'
        if lof_roc_auc_key in methods:
            performance_data_roc_auc.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'LOF',
                'Metric': 'ROC AUC',
                'Score': methods[lof_roc_auc_key]
            })

# Convert the list of performance data into a Pandas DataFrame
roc_auc_df_tuned = pd.DataFrame(performance_data_roc_auc)

# Create a grouped bar chart using seaborn.catplot for ROC AUC
g_roc_auc = sns.catplot(data=roc_auc_df_tuned, x='Attack Type', y='Score', hue='Model', col='Feature Selection',
                kind='bar', palette='viridis', dodge=True, col_wrap=3, height=5, aspect=1.2)

# Add appropriate labels and a title
g_roc_auc.set_axis_labels('Attack Type', 'ROC AUC Score')
g_roc_auc.set_titles('Feature Selection: {col_name}')
g_roc_auc.set_xticklabels(rotation=45, ha='right')
plt.suptitle('Tuned Model ROC AUC Score Comparison Across Attack Types, Feature Selection, and Models', y=1.02, fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap

# Display the plot
plt.show()


**Reasoning**:
I will now update the grouped bar charts to include the Average Precision (AP) for the newly evaluated OCSVM and LOF models, alongside Isolation Forest and Autoencoder. This will provide a comprehensive visualization of all models' performance after tuning, addressing the second part of the visualization subtask.



**Reasoning**:
The previous code failed because of a typo in the seaborn function call. I will correct `sns.catcatplot` to `sns.catplot` to resolve the `AttributeError`.



In [ ]:
print('--- All Model Performance Metrics (ROC AUC and Average Precision) ---')
display(df_fixed.sort_values(by=['attack_type', 'model', 'feature_selection']).reset_index(drop=True))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize an empty list to store performance data for Average Precision
performance_data_avg_precision = []

# Define selection method prefixes for iteration consistency
selection_method_prefixes = ['heuristic', 'variance', 'pca']

# Iterate through each attack_type and its methods in the results dictionary
for attack_type, methods in results.items():
    # For each attack_type, iterate through the selection_method_prefixes
    for prefix in selection_method_prefixes:
        # Check for Isolation Forest Average Precision scores (tuned)
        if_avg_precision_key = f'{prefix}_if_avg_precision_tuned'
        if if_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Isolation Forest',
                'Metric': 'Average Precision',
                'Score': methods[if_avg_precision_key]
            })

        # Check for Autoencoder Average Precision scores (tuned - using previous values)
        ae_avg_precision_key = f'{prefix}_ae_avg_precision_tuned'
        if ae_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'Autoencoder',
                'Metric': 'Average Precision',
                'Score': methods[ae_avg_precision_key]
            })

        # Check for OCSVM Average Precision scores (tuned)
        ocsvm_avg_precision_key = f'{prefix}_ocsvm_avg_precision_tuned'
        if ocsvm_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'OCSVM',
                'Metric': 'Average Precision',
                'Score': methods[ocsvm_avg_precision_key]
            })

        # Check for LOF Average Precision scores (tuned)
        lof_avg_precision_key = f'{prefix}_lof_avg_precision_tuned'
        if lof_avg_precision_key in methods:
            performance_data_avg_precision.append({
                'Attack Type': attack_type.replace('_', ' ').title(),
                'Feature Selection': prefix.title(),
                'Model': 'LOF',
                'Metric': 'Average Precision',
                'Score': methods[lof_avg_precision_key]
            })

# Convert the list of performance data into a Pandas DataFrame
avg_precision_df_tuned = pd.DataFrame(performance_data_avg_precision)

# Create a grouped bar chart using seaborn.catplot for Average Precision
g_avg_precision = sns.catplot(data=avg_precision_df_tuned, x='Attack Type', y='Score', hue='Model', col='Feature Selection',
                kind='bar', palette='viridis', dodge=True, col_wrap=3, height=5, aspect=1.2)

# Add appropriate labels and a title
g_avg_precision.set_axis_labels('Attack Type', 'Average Precision Score')
g_avg_precision.set_titles('Feature Selection: {col_name}')
g_avg_precision.set_xticklabels(rotation=45, ha='right')
plt.suptitle('Tuned Model Average Precision Score Comparison Across Attack Types, Feature Selection, and Models', y=1.02, fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap

# Display the plot
plt.show()

**Reasoning**:
I will update the visualization for the best performing model. First, I need to iterate through the `results` dictionary to find the model (Isolation Forest, Autoencoder, OCSVM, or LOF) that achieved the highest ROC AUC score after tuning across all attack types and feature selection methods. Then, I will retrieve the relevant test scores (anomaly scores or reconstruction errors) and the corresponding ground truth labels for this best model and create a scatter plot to visualize its performance.



In [ ]:
# Recompute and fix score orientation + evaluate consistently
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np
import pandas as pd
import traceback

fixed_summary = []

def safe_transform(scaler, X):
    try:
        return scaler.transform(X)
    except Exception:
        return X

for attack_type, methods in results.items():
    for prefix in ['heuristic', 'variance', 'pca']:
        try:
            X_test = methods.get(f'{prefix}_X_test')
            y_test = methods.get(f'{prefix}_y_test')
            if X_test is None or y_test is None or len(X_test) == 0:
                continue
            if len(np.unique(y_test)) < 2:
                # Can't compute ROC/AP with single class
                continue

            # Prefer prefix-level scaler if stored, fallback to several possible keys
            scaler = methods.get(f'{prefix}_scaler') or methods.get(f'{prefix}_lof_scaler') or methods.get('scaler') or None
            Xs = safe_transform(scaler, X_test) if scaler is not None else X_test

            # Helper to evaluate model whose decision_function/score_samples returns higher for inliers
            def eval_model_score(model, Xs, invert=True):
                if model is None:
                    return None, None
                # get score array (higher = inlier by convention for these APIs)
                if hasattr(model, 'decision_function'):
                    scores = model.decision_function(Xs)
                elif hasattr(model, 'score_samples'):
                    scores = model.score_samples(Xs)
                else:
                    return None, None
                # Make anomaly_score such that higher = more anomalous
                anomaly_score = -scores if invert else scores
                roc = roc_auc_score(y_test, anomaly_score)
                ap = average_precision_score(y_test, anomaly_score)
                return roc, ap

            # 1) Isolation Forest (use tuned if available)
            if_model = methods.get(f'{prefix}_if_tuned_model') or methods.get(f'{prefix}_isolation_forest_model') or methods.get('isolation_forest_model')
            if roc_ap := eval_model_score(if_model, Xs, invert=True):
                roc, ap = roc_ap
                results[attack_type][f'{prefix}_if_roc_auc_fixed'] = roc
                results[attack_type][f'{prefix}_if_ap_fixed'] = ap
                fixed_summary.append((attack_type, prefix, 'IsolationForest', roc, ap))

            # 2) OCSVM
            ocsvm = methods.get(f'{prefix}_ocsvm_tuned_model') or methods.get(f'{prefix}_ocsvm_model') or methods.get('ocsvm_model')
            if roc_ap := eval_model_score(ocsvm, Xs, invert=True):
                roc, ap = roc_ap
                results[attack_type][f'{prefix}_ocsvm_roc_auc_fixed'] = roc
                results[attack_type][f'{prefix}_ocsvm_ap_fixed'] = ap
                fixed_summary.append((attack_type, prefix, 'OCSVM', roc, ap))

            # 3) LOF
            lof = methods.get(f'{prefix}_lof_tuned_model') or methods.get(f'{prefix}_lof_model') or methods.get('lof_model')
            if roc_ap := eval_model_score(lof, Xs, invert=True):
                roc, ap = roc_ap
                results[attack_type][f'{prefix}_lof_roc_auc_fixed'] = roc
                results[attack_type][f'{prefix}_lof_ap_fixed'] = ap
                fixed_summary.append((attack_type, prefix, 'LOF', roc, ap))

            # 4) Autoencoder (reconstruction error: higher = more anomalous)
            ae_errors = methods.get(f'{prefix}_test_reconstruction_errors')
            if ae_errors is not None and len(ae_errors) == len(y_test):
                ae_roc = roc_auc_score(y_test, ae_errors)
                ae_ap = average_precision_score(y_test, ae_errors)
                results[attack_type][f'{prefix}_ae_roc_auc_fixed'] = ae_roc
                results[attack_type][f'{prefix}_ae_ap_fixed'] = ae_ap
                fixed_summary.append((attack_type, prefix, 'Autoencoder', ae_roc, ae_ap))
        except Exception:
            print(f'Error evaluating {attack_type} - {prefix}:')
            traceback.print_exc()

# Print a tidy DataFrame of the fixed metrics
df_fixed = pd.DataFrame(fixed_summary, columns=['attack_type', 'feature_selection', 'model', 'roc_auc', 'avg_precision'])
if df_fixed.empty:
    print('No re-evaluated metrics generated (maybe many y_test arrays had single class).')
else:
    print(df_fixed.sort_values(['attack_type', 'model', 'feature_selection']).to_string(index=False))

In [ ]:
# Diagnostic: show sample sizes and class balance per config
import pandas as pd, numpy as np

rows = []
for attack_type, methods in results.items():
    for prefix in ['heuristic','variance','pca']:
        X_train = methods.get(f'{prefix}_X_train')
        X_val = methods.get(f'{prefix}_X_val')
        X_test = methods.get(f'{prefix}_X_test')
        y_train = methods.get(f'{prefix}_y_train')
        y_val = methods.get(f'{prefix}_y_val')
        y_test = methods.get(f'{prefix}_y_test')
        rows.append({
            'attack_type': attack_type,
            'prefix': prefix,
            'n_train': None if X_train is None else len(X_train),
            'n_val': None if X_val is None else len(X_val),
            'n_test': None if X_test is None else len(X_test),
            'anoms_train': None if y_train is None else int(np.sum(y_train==1)),
            'anoms_val': None if y_val is None else int(np.sum(y_val==1)),
            'anoms_test': None if y_test is None else int(np.sum(y_test==1)),
        })
pd.DataFrame(rows).sort_values(['attack_type','prefix']).reset_index(drop=True)

# Task
The previous execution in cell `ns3ewJZkejKZ` failed to find any features for splitting, leading to subsequent models also failing or being skipped. This was due to two issues in that cell:
1.  It incorrectly reset the engineered feature DataFrames (`user_features`, `geo_features`, `priv_user_features`, `port_scan_features`) back to the raw `df` before feature selection. These DataFrames should retain their engineered state from earlier cells.
2.  It lacked the core logic to perform heuristic, variance-based, and PCA-based feature selections and populate the `results` dictionary, as intended by the plan.

To resolve this, I will perform the following actions:
1.  **Modify cell `ns3ewJZkejKZ`**:
    *   Remove the lines that overwrite the `user_features`, `geo_features`, `priv_user_features`, and `port_scan_features` DataFrames.
    *   Remove the redundant ground truth DataFrame definitions as these are handled correctly in subsequent cells (`r6vK8FaCIdJh`).
    *   Insert the feature selection code into the designated placeholder, which will use the already engineered feature DataFrames to populate the `results` dictionary with selected features, scaled feature arrays, and scaler objects for each attack type and selection method.

This ensures that the feature engineering performed earlier is preserved and that the `results` dictionary is correctly populated, allowing `rebuild_stratified_splits()` to function as intended.

```python
# Clear everything and start fresh
import pandas as pd
import numpy as np

print("=== STARTING FRESH WITH CONSISTENT DATA ===")

# 1. Make sure df is your 500K dataset
print(f"Using df with {df.shape[0]:,} rows, {df.shape[1]} columns")

# Removed: 2. Reset all feature DataFrames to df.
# The DataFrames `user_features`, `geo_features`, `priv_user_features`, and `port_scan_features`
# are expected to be populated with engineered features from previous cells.

# Removed: Ground truth DataFrame definitions are handled globally in other cells.


print("\n" + "="*60)
print("Running feature selection with consistent data...")
print("="*60)

# Clear old results
results = {}

# --- START OF INSERTED FEATURE SELECTION CODE ---
attack_types_list = ['brute_force', 'geolocation', 'privilege', 'port_scan']

for attack_type in attack_types_list:
    results[attack_type] = {} # Initialize for this attack type

    # Determine which base DataFrame to use for feature selection
    # These DataFrames are expected to have been engineered in previous cells.
    if attack_type == 'brute_force':
        base_df = user_features
    elif attack_type == 'geolocation':
        base_df = geo_features
    elif attack_type == 'privilege':
        base_df = priv_user_features
    elif attack_type == 'port_scan':
        base_df = port_scan_features
    else:
        print(f"Warning: Unknown attack type {attack_type}, skipping feature selection.")
        continue

    print(f"  Performing feature selection for {attack_type} (using {base_df.shape[0]:,} rows)...")

    # Heuristic selection
    heuristic_cols = select_features_by_attack_type(attack_type)
    if heuristic_cols:
        X_array, scaler_obj, cols_used = prepare_selected_features(base_df, heuristic_cols)
        results[attack_type]['heuristic_cols'] = cols_used
        results[attack_type]['heuristic_X_full'] = X_array
        results[attack_type]['heuristic_scaler'] = scaler_obj
        print(f"    Heuristic: {len(cols_used)} features selected. Shape: {X_array.shape}")
    else:
        print(f"    Heuristic: No columns found for {attack_type}.")

    # Variance-based selection
    variance_cols = variance_based_selection(base_df, top_k=10, corr_threshold=0.9)
    if variance_cols:
        X_array, scaler_obj, cols_used = prepare_selected_features(base_df, variance_cols)
        results[attack_type]['variance_cols'] = cols_used
        results[attack_type]['variance_X_full'] = X_array
        results[attack_type]['variance_scaler'] = scaler_obj
        print(f"    Variance: {len(cols_used)} features selected. Shape: {X_array.shape}")
    else:
        print(f"    Variance: No columns found for {attack_type}.")

    # PCA-based selection
    pca_cols = pca_loadings_selection(base_df, n_components=5, top_k=10)
    if pca_cols:
        X_array, scaler_obj, cols_used = prepare_selected_features(base_df, pca_cols)
        results[attack_type]['pca_cols'] = cols_used
        results[attack_type]['pca_X_full'] = X_array
        results[attack_type]['pca_scaler'] = scaler_obj
        print(f"    PCA: {len(cols_used)} features selected. Shape: {X_array.shape}")
    else:
        print(f"    PCA: No columns found for {attack_type}.")

# --- END OF INSERTED FEATURE SELECTION CODE ---


# 5. THEN run the stratified split
print("\n" + "="*60)
print("Running stratified splits...")
print("="*60)

# Run the rebuild_stratified_splits() function
# This function expects user_ground_truth_df and port_scan_ground_truth_df to be globally defined
# from prior cells like r6vK8FaCIdJh.
results = rebuild_stratified_splits()

print("\n✅ DONE! You should now have thousands of anomalies per split.")
```

## Insert Feature Selection Code

### Subtask:
Modify cell `ns3ewJZkejKZ` to correctly perform heuristic, variance-based, and PCA-based feature selections and populate the `results` dictionary.


**Reasoning**:
The subtask requires modifying cell `ns3ewJZkejKZ` to correctly perform feature selection and populate the `results` dictionary. This involves removing the incorrect re-initialization of feature DataFrames and ground truth DataFrames, and inserting the feature selection logic that uses the already engineered feature DataFrames (`user_features`, `geo_features`, `priv_user_features`, `port_scan_features`) from previous steps. I will also add checks to ensure selected columns actually exist in the DataFrames and will populate the `results` dictionary with `_full_X` and corresponding `_scaler` for each method, as `rebuild_stratified_splits()` will later handle the train/val/test splits.



In [ ]:
# Clear everything and start fresh
import pandas as pd
import numpy as np

print("=== STARTING FEATURE SELECTION PROCESS ===")

# The feature DataFrames (user_features, geo_features, priv_user_features, port_scan_features)
# are expected to be populated from previous cells (e.g., 4Ng7N4I7SXyr, OxI_SUmNoXpA, tDZ6Ig3hZI3c, TUWMpm3mZRLe).
# We explicitly avoid resetting them here to preserve the engineered features.

print("\n" + "="*60)
print("Running feature selection...")
print("="*60)

# Clear old results dictionary for fresh population
results = {}

# Iterate through each attack type and perform feature selection
for attack_type, feature_df_name in [
    ('brute_force', 'user_features'),
    ('geolocation', 'geo_features'),
    ('privilege', 'priv_user_features'),
    ('port_scan', 'port_scan_features')
]:
    # Get the correct feature DataFrame based on its global variable name
    feature_df = globals()[feature_df_name]

    print(f"\nProcessing feature selection for: {attack_type} (using {feature_df_name})")

    results[attack_type] = {} # Initialize results for this specific attack type

    # --- Heuristic Selection ---
    heuristic_cols = select_features_by_attack_type(attack_type)
    # Ensure all heuristic columns exist in the DataFrame before proceeding
    if heuristic_cols and all(col in feature_df.columns for col in heuristic_cols):
        results[attack_type]['heuristic_cols'] = heuristic_cols
        X_heuristic, scaler_heuristic, _ = prepare_selected_features(feature_df, heuristic_cols)
        # Store full X and scaler for heuristic method for this attack type
        results[attack_type]['heuristic_full_X'] = X_heuristic
        results[attack_type]['heuristic_scaler'] = scaler_heuristic
        print(f"  Heuristic selected {len(heuristic_cols)} features. X_full shape: {X_heuristic.shape}")
    else:
        print(f"  Heuristic selection found no valid columns or some columns are missing for {attack_type}.")
        results[attack_type]['heuristic_cols'] = [] # Ensure it's an empty list if no valid columns found


    # --- Variance-based Selection ---
    # Select top features by variance, then de-correlate
    variance_cols = variance_based_selection(feature_df, top_k=10, corr_threshold=0.9)
    # Ensure all variance columns exist in the DataFrame before proceeding
    if variance_cols and all(col in feature_df.columns for col in variance_cols):
        results[attack_type]['variance_cols'] = variance_cols
        X_variance, scaler_variance, _ = prepare_selected_features(feature_df, variance_cols)
        results[attack_type]['variance_full_X'] = X_variance
        results[attack_type]['variance_scaler'] = scaler_variance
        print(f"  Variance selected {len(variance_cols)} features. X_full shape: {X_variance.shape}")
    else:
        print(f"  Variance selection found no valid columns or some columns are missing for {attack_type}.")
        results[attack_type]['variance_cols'] = [] # Ensure it's an empty list if no valid columns found

    # --- PCA-based Selection ---
    # Select top features based on PCA loadings
    pca_cols = pca_loadings_selection(feature_df, n_components=5, top_k=10)
    # Ensure all PCA columns exist in the DataFrame before proceeding
    if pca_cols and all(col in feature_df.columns for col in pca_cols):
        results[attack_type]['pca_cols'] = pca_cols
        X_pca, scaler_pca, _ = prepare_selected_features(feature_df, pca_cols)
        results[attack_type]['pca_full_X'] = X_pca
        results[attack_type]['pca_scaler'] = scaler_pca
        print(f"  PCA selected {len(pca_cols)} features. X_full shape: {X_pca.shape}")
    else:
        print(f"  PCA selection found no valid columns or some columns are missing for {attack_type}.")
        results[attack_type]['pca_cols'] = [] # Ensure it's an empty list if no valid columns found

print("\nFeature selection complete. Results dictionary populated.")

# The creation of user_ground_truth_df and port_scan_ground_truth_df,
# along with the initial stratified split, will be handled by rebuild_stratified_splits()
# in the next step, using the original df and its 'anomaly_label' column.

print("\n" + "="*60)
print("Running stratified splits...")
print("="*60)

# Run the rebuild_stratified_splits() function, which will now use the 'results' dictionary
# populated above with 'full_X' data for each selection method.
results = rebuild_stratified_splits()

print("\n✅ DONE! You should now have thousands of anomalies per split.")

### Addressing the "Heuristic selection found no valid columns" Issue

The previous execution showed that "Heuristic selection found no valid columns or some columns are missing" for all attack types, and that Variance and PCA selected only 3 features. This indicates that the DataFrames `user_features`, `geo_features`, `priv_user_features`, and `port_scan_features` were not correctly populated with the engineered features before cell `ns3ewJZkejKZ` was executed. Instead, they likely contained only the original `df` columns.

To fix this, you need to re-run the cells that perform feature engineering for each attack type. Please go back and re-execute the following code cells in your notebook in order:

*   **`4Ng7N4I7SXyr`**: Generates `user_features` (for brute-force attacks)
*   **`OxI_SUmNoXpA`**: Generates `geo_features` (for geolocation attacks)
*   **`tDZ6Ig3hZI3c`**: Generates `port_scan_features` (for port scanning attacks) - *Note: This cell might need `df_by_source` to be correctly defined from earlier in the notebook if `port_scan_features` was previously overwritten incorrectly.*
*   **`TUWMpm3mZRLe`**: Generates `priv_user_features` (for privilege escalation attacks)

After re-running these cells to ensure the feature DataFrames are correctly populated with engineered features, you should then re-execute cell `ns3ewJZkejKZ`.

### Addressing the "Heuristic selection found no valid columns" Issue

The previous execution showed that "Heuristic selection found no valid columns or some columns are missing" for all attack types, and that Variance and PCA selected only 3 features. This indicates that the DataFrames `user_features`, `geo_features`, `priv_user_features`, and `port_scan_features` were not correctly populated with the engineered features before cell `ns3ewJZkejKZ` was executed. Instead, they likely contained only the original `df` columns.

To fix this, you need to re-run the cells that perform feature engineering for each attack type. Please go back and re-execute the following code cells in your notebook in order:

*   **`4Ng7N4I7SXyr`**: Generates `user_features` (for brute-force attacks)
*   **`OxI_SUmNoXpA`**: Generates `geo_features` (for geolocation attacks)
*   **`tDZ6Ig3hZI3c`**: Generates `port_scan_features` (for port scanning attacks) - *Note: This cell might need `df_by_source` to be correctly defined from earlier in the notebook if `port_scan_features` was previously overwritten incorrectly.*
*   **`TUWMpm3mZRLe`**: Generates `priv_user_features` (for privilege escalation attacks)

After re-running these cells to ensure the feature DataFrames are correctly populated with engineered features, you should then re-execute cell `ns3ewJZkejKZ`.

## Re-execute Feature Engineering Cells

### Subtask:
Re-execute the cells responsible for feature engineering to ensure `user_features`, `geo_features`, `port_scan_features`, and `priv_user_features` DataFrames are correctly populated.


**Reasoning**:
The subtask requires re-executing cell `4Ng7N4I7SXyr` to ensure the `user_features` DataFrame is correctly populated.



In [ ]:
# User-level aggregated features
user_features = df.groupby('username').agg({
    'timestamp': ['count', 'min', 'max'],  # Total activities, first & last login
    'status': lambda x: (x == 'Failed').sum(),  # Total failures
    'attempts': ['sum', 'mean', 'max'],  # Attempt patterns
    'source_ip': 'nunique',  # Unique IPs used
    'service': 'nunique',  # Unique services accessed
    'server': 'nunique',  # Unique servers accessed
    'port': 'nunique'  # Unique ports used
}).reset_index()

# Flatten the column names
user_features.columns = ['username', 'total_activities', 'first_activity', 'last_activity',
                        'total_failures', 'total_attempts', 'avg_attempts', 'max_attempts',
                        'unique_ips', 'unique_services', 'unique_servers', 'unique_ports']

# Calculate derived features
user_features['failure_rate'] = user_features['total_failures'] / user_features['total_activities']
user_features['activity_duration_days'] = (user_features['last_activity'] - user_features['first_activity']).dt.total_seconds() / (24 * 3600)
user_features['activities_per_day'] = user_features['total_activities'] / user_features['activity_duration_days'].replace(0, 1)

**Reasoning**:
Continuing to address the subtask, the next step is to execute cell `OxI_SUmNoXpA` to generate the `geo_features` DataFrame, ensuring it's correctly populated.



In [ ]:
# Separate aggregations by data type
basic_features = df_geo.groupby('username').agg({
    'server': 'nunique',
    'source_ip': 'nunique',
    'service': 'nunique',
    'timestamp': ['count', 'min', 'max'],
}).reset_index()
basic_features.columns = ['username', 'unique_servers', 'unique_ips', 'unique_services', 'total_logins', 'first_seen', 'last_seen']

# For numeric features - handle NaN during aggregation
numeric_features = df_geo.groupby('username').agg({
    'time_since_last_login_hours': ['mean', 'std', 'min', 'max'],
    'server_access_count': ['mean', 'max'],
    'ip_access_count': ['mean', 'max'],
    'attempts': ['mean', 'max', 'sum'],
    'port': 'nunique',
}).reset_index()
numeric_features.columns = ['username', 'avg_time_between_logins', 'std_time_between_logins', 'min_time_between_logins',
                           'max_time_between_logins', 'avg_server_access', 'max_server_access', 'avg_ip_access',
                           'max_ip_access', 'avg_attempts', 'max_attempts', 'total_attempts', 'unique_ports']

# Boolean features
boolean_features = df_geo.groupby('username').agg({
    'server_change': 'sum',
    'ip_change': 'sum',
    'consecutive_same_location': 'max',
}).reset_index()
boolean_features.columns = ['username', 'server_changes', 'ip_changes', 'max_consecutive_same_location']

# Status features
status_features = df_geo.groupby('username').agg({
    'status': lambda x: (x == 'failure').sum(),
}).reset_index()
status_features.columns = ['username', 'failed_logins']

# Merge all features
geo_features = basic_features.merge(numeric_features, on='username', how='left')
geo_features = geo_features.merge(boolean_features, on='username', how='left')
geo_features = geo_features.merge(status_features, on='username', how='left')

#handle devision by zero
geo_features['active_days'] = (
    (geo_features['last_seen'] - geo_features['first_seen']).dt.total_seconds() / (24 * 3600)
).replace(0, 1)  # Avoid division by zero

# Safe division to prevent NaN
geo_features['logins_per_day'] = np.where(
    geo_features['active_days'] > 0,
    geo_features['total_logins'] / geo_features['active_days'],
    0
)
geo_features['servers_per_login'] = np.where(
    geo_features['total_logins'] > 0,
    geo_features['unique_servers'] / geo_features['total_logins'],
    0
)
geo_features['ips_per_login'] = np.where(
    geo_features['total_logins'] > 0,
    geo_features['unique_ips'] / geo_features['total_logins'],
    0
)
geo_features['failure_rate'] = np.where(
    geo_features['total_logins'] > 0,
    geo_features['failed_logins'] / geo_features['total_logins'],
    0
)

# Handle NaN in std_time_between_logins (occurs when users have only one login)
geo_features['std_time_between_logins'] = geo_features['std_time_between_logins'].fillna(0)

# Risk scores
geo_features['location_volatility'] = np.where(
    geo_features['total_logins'] > 0,
    (geo_features['server_changes'] + geo_features['ip_changes']) / geo_features['total_logins'],
    0
)

geo_features['geo_risk_score'] = (
    geo_features['unique_ips'] * 0.3 +
    geo_features['unique_servers'] * 0.3 +
    geo_features['location_volatility'] * 0.2 +
    geo_features['failure_rate'] * 0.2
)

# Fill any remaining NaN values with median
numeric_cols = geo_features.select_dtypes(include=np.number).columns
geo_features[numeric_cols] = geo_features[numeric_cols].fillna(geo_features[numeric_cols].median())

print(f"\nRemaining NaN values in geo_features: {geo_features.isnull().sum().sum()}")

**Reasoning**:
Continuing to address the subtask, the next step is to execute cell `tDZ6Ig3hZI3c` to generate the `port_scan_features` DataFrame, ensuring it's correctly populated.



In [ ]:
port_scan_features = df_by_source.groupby(['source_ip', 'time_window']).agg({
    'port': [
        'count',  # total_port_attempts
        'nunique',  # unique_ports_scanned
        lambda x: (x > 10000).sum(),  # high_port_attempts (verify threshold)
        'min',  # min_port
        'max',  # max_port
        lambda x: np.std(x) if len(x) > 1 else 0,  # port_std_dev
    ],
    'status': [
        lambda x: (x == 'Failed').sum(),  # failed_attempts
        lambda x: (x == 'Success').sum(),  # success_attempts
    ],
    'server': 'nunique',  # unique_servers_targeted
    'service': 'nunique',  # unique_services_accessed
    'username': 'nunique',  # unique_usernames_tried
}).reset_index()


**Reasoning**:
Continuing to address the subtask, the next step is to execute cell `tDZ6Ig3hZI3c` to generate the `port_scan_features` DataFrame, ensuring it's correctly populated.



In [ ]:
port_scan_features = df_by_source.groupby(['source_ip', 'time_window']).agg({
    'port': [
        'count',  # total_port_attempts
        'nunique',  # unique_ports_scanned
        lambda x: (x > 10000).sum(),  # high_port_attempts (verify threshold)
        'min',  # min_port
        'max',  # max_port
        lambda x: np.std(x) if len(x) > 1 else 0,  # port_std_dev
    ],
    'status': [
        lambda x: (x == 'Failed').sum(),  # failed_attempts
        lambda x: (x == 'Success').sum(),  # success_attempts
    ],
    'server': 'nunique',  # unique_servers_targeted
    'service': 'nunique',  # unique_services_accessed
    'username': 'nunique',  # unique_usernames_tried
}).reset_index()
